Notebook for the paper 'Skill and spatial mismatches for sustainable development in Brazil' 2025


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import matplotlib as mpl
# import geopandas as gpd

In [ ]:
data_path = 'data/'

In [ ]:
## omng with only CC component and non-0 2018-emp has 6496 occ-reg pairs
omng_df = pd.read_csv(data_path+'omng_merged_connected_component.csv', dtype = {'Unnamed: 0': np.str_}
                     ).rename(columns={'Unnamed: 0': ''}).set_index('')
omng_df.head()

In [ ]:
scenarios = ['Cen2BAseq', 'Cen2Aseq', 'Cen5Aseq']

In [ ]:
scenario_dict = {scn: pd.read_csv(data_path+scn+'scenario_dict_mcc.csv', index_col='Unnamed: 0') 
                 for scn in scenarios}

In [ ]:
scenario_dict['Cen2BAseq']

In [ ]:
cbo_occ_titles_merge1 = pd.read_csv(data_path+'merged_names_408.csv', index_col='Unnamed: 0')

In [ ]:
cbo_occ_titles_merge1.loc[-1] = ['0XXX', 'Military workers']  # adding a row
cbo_occ_titles_merge1.index = cbo_occ_titles_merge1.index + 1  # shifting index
cbo_occ_titles_merge1 = cbo_occ_titles_merge1.sort_index()  # sorting by index
    
cbo_occ_titles_merge1.head()

In [ ]:
years = scenario_dict['Cen2BAseq'].index
regions = list(np.unique([occ[4:] for occ in scenario_dict['Cen2BAseq'].columns]))

occs = list(set([occ[:4] for occ in omng_df.columns]))
occs.sort() ## military occupations removed

len(occs)

occs_regions_list = omng_df.columns

scn_labels = {scn: name for scn, name in zip(scenarios, ['Baseline', 'Agriculture growth path', 
                                                         'Manufacturing growth path',
                                                        ])}

In [ ]:
yr_sums = {scn: scenario_dict[scn].sum(axis=1) for scn in scenarios}

In [ ]:
scenario_dict_norm_to_bl = {}
L = scenario_dict['Cen2BAseq'].sum(axis=1)[2018]
# # L = L*1.0165
for scn in scenarios:
    scenario_dict_norm_to_bl[scn] = (scenario_dict[scn].div(
                                        scenario_dict['Cen2BAseq'].sum(axis=1), axis=0))*L
    

In [ ]:
yr_sums_norm_to_bl = {scn: scenario_dict_norm_to_bl[scn].sum(axis=1) for scn in scenarios}

In [ ]:
scn = 'Cen2BAseq'
employment = np.array(scenario_dict_norm_to_bl[scn].loc[2018].values, dtype=np.float64)
# wage = np.random.rand(n)*1e5
print(len(employment))

In [ ]:
large_occs = list(np.where(employment>= 0)[0])

In [ ]:
def make_F_networks(T_networks):
    '''makes a list of flow adjacency matrices based on a list
    of Transitions matrices
    '''
    # create list where adjacency matrix will be appended
    L_networks = []
    for i, T in enumerate(T_networks):
        # if there were no out transitions just set flow equal to zero
        with np.errstate(divide='ignore', invalid='ignore'):
            F = T/T.sum(axis=1, keepdims=True)
        F[np.isnan(F)] = 0
        L_networks.append(F)
        
    return L_networks

In [ ]:
δ_u = 0.016
δ_v = 0.012
γ_u = 10*δ_u
γ_v = γ_u
τ = 7 # time steps after which worker is long-term unemployed

# occupational mobility network
T_omn = np.array(omng_df)
A_omn = make_F_networks([T_omn])[0]
n = T_omn.shape[0]
# A_omn = add_self_loops(A_omn, r)
# complete network
A_kn = np.ones([n,n])/n

# shock and time conditions
# NOTE one time step ~ 4 weeks
# t_shock = 200 + (9 * 52/4) # time at which shock starts i.e. end of 2018
t_shock = 20 * 52/4 # time at which shock starts i.e. end of 2018
t_simulation = 500
shock_duration = 12 * 52/4
time_array = [t*4/52 for t in range(t_simulation)]

In [ ]:
# initial employment + vacancies = 2018 demand from CGE model
employment_0 = (1-δ_v)*employment[:]
unemployment_0 = δ_u * employment_0
vacancies_0 = δ_v * employment_0
# labor force is all workers, employed + unemployed
L = np.sum(employment_0 + unemployment_0)

# initial demand and target demand
D_0 = employment_0 + vacancies_0

In [ ]:
def target_demand_piecewise_linear(t, d_0, scn_dict, scn=scn, t_shock=195, shock_duration=12*(52/4)):
    """function that creates numpy array of monthly target demand for each occupation
    Args: 
        scenario: pandasDataFrame
    Returns:
        d_dagger: demand of occupation at time t"""
    if t < t_shock:
        return d_0
    elif t < t_shock+shock_duration:
        month = t % 13
        
        cur_yr = int(2019 + np.floor((t-t_shock) / (52/4)))
        # spread increase from cur_yr-1 to cur_yr over the 13 4-week periods
#         print(cur_yr)
        four_wk_change = (scn_dict[scn].loc[cur_yr]-scn_dict[scn].loc[cur_yr-1])/(52/4)
        
        d_dagger = np.array(scn_dict[scn].loc[cur_yr-1], dtype=np.float64
                           )+(np.array(four_wk_change, dtype=np.float64)*month)
        return d_dagger
    else:
        d_dagger = np.array(scn_dict[scn].loc[2030], dtype=np.float64)
        return d_dagger

In [ ]:
def percentage_change_u_to_baseline(E, U, E_b, U_b, time_start_1, time_end_1):
    u_scn_num = period_occ_u_rate(E, U, time_start_1, time_end_1)
    u_baseline_num = period_occ_u_rate(E_b, U_b, time_start_1, time_end_1)
    return np.array(100*(u_scn_num-u_baseline_num) / u_baseline_num)
def period_occ_u_rate(E, U, time_start, time_end):
        """ gives the period unemployment rate of an occupation
        E: employment rate of occupation per time
        U: unemployment rate of occupation per time
        time_start(int): time from which the average starts
        time_end(int): time from which the average ends
        """
        e = E[time_start:time_end, :]
        u = U[time_start:time_end, :]
        return 100*sum(u) / sum(e + u)
def percentage_POINT_change_u_to_baseline(E, U, E_b, U_b, time_start_1, time_end_1):
    u_scn_num = period_occ_u_rate(E, U, time_start_1, time_end_1)
    u_baseline_num = period_occ_u_rate(E_b, U_b, time_start_1, time_end_1)
    return u_scn_num - u_baseline_num

In [ ]:
def percentage_change_u(E, U, time_start_1, time_end_1, time_start_2, time_end_2):
    u_rate_start = period_occ_u_rate(E, U, time_start_1, time_end_1)
    u_rate_end = period_occ_u_rate(E, U, time_start_2, time_end_2)
    return np.array(100*(u_rate_end-u_rate_start) / u_rate_start)
def percentage_POINT_change_u(E, U, time_start_1, time_end_1, time_start_2, time_end_2):
    u_rate_start = period_occ_u_rate(E, U, time_start_1, time_end_1)
    u_rate_end = period_occ_u_rate(E, U, time_start_2, time_end_2)
    return u_rate_end - u_rate_start

In [ ]:
occ_titles_dict = {row['CODE']: row['TITLE'] for idx, row 
                   in cbo_occ_titles_merge1.iterrows()}

In [ ]:
def u_longterm_from_jobspell(U_ltm, τ):
    return np.sum(U_ltm[:, τ-1:, :], axis=1)

In [ ]:
## agent-based model import
import labornet as lbn

In [ ]:
## create 'geog_results' folder in your data_path folder

In [ ]:
# results_dict_norm_to_bl = {}
for scn in scenarios:
    print(scn)
    results_dict_norm_to_bl[scn] = {}
    [E, U, V], _, D, V_all = lbn.run_numerical_solution(lbn.fire_and_hire_workers, \
    t_simulation, [δ_u, δ_v, γ_u, γ_v], [employment_0, unemployment_0, vacancies_0], \
    target_demand_piecewise_linear, employment, scenario_dict_norm_to_bl, scn, t_shock, shock_duration, \
    lbn.matching_probability, A_omn, τ)
    
#     results_dict_norm_to_bl[scn]['E'] = E
#     results_dict_norm_to_bl[scn]['U'] = U
#     results_dict_norm_to_bl[scn]['V'] = V
#     results_dict_norm_to_bl[scn]['D'] = D
    np.savetxt('data/geog_results/results_omng_'+scn+'_E.csv', E, delimiter=',')
    np.savetxt('data/geog_results/results_omng_'+scn+'_U.csv', U, delimiter=',')
    np.savetxt('data/geog_results/results_omng_'+scn+'_V.csv', V, delimiter=',')
    V_lt = u_longterm_from_jobspell(V_all, 6)
#     results_dict_norm_to_bl[scn]['V_lt'] = V_lt

    np.savetxt('data/geog_results/results_omng_'+scn+'_V_lt.csv', V_lt, delimiter=',')
    np.savetxt('data/geog_results/results_omng_'+scn+'_D.csv', D, delimiter=',')

In [ ]:
# results_dict_norm_kn_to_bl = {}
for scn in scenarios:
    print(scn)
#     results_dict_norm_kn_to_bl[scn] = {}
    [E, U, V], _, D, V_all = lbn.run_numerical_solution(lbn.fire_and_hire_workers, \
    t_simulation, [δ_u, δ_v, γ_u, γ_v], [employment_0, unemployment_0, vacancies_0], \
    target_demand_piecewise_linear, employment, scenario_dict_norm_to_bl, scn, t_shock, shock_duration, \
    lbn.matching_probability, A_kn, τ)
    
#     results_dict_norm_kn_to_bl[scn]['E'] = E
#     results_dict_norm_kn_to_bl[scn]['U'] = U
#     results_dict_norm_kn_to_bl[scn]['V'] = V
#     results_dict_norm_kn_to_bl[scn]['V_all'] = V_all
#     results_dict_norm_kn_to_bl[scn]['D'] = D
    np.savetxt('data/geog_results/results_kn_'+scn+'_E.csv', E, delimiter=',')
    np.savetxt('data/geog_results/results_kn_'+scn+'_U.csv', U, delimiter=',')
    np.savetxt('data/geog_results/results_kn_'+scn+'_V.csv', V, delimiter=',')
    V_lt = u_longterm_from_jobspell(V_all, 6)
    np.savetxt('data/geog_results/results_kn_'+scn+'_V_lt.csv', V_lt, delimiter=',')
    np.savetxt('data/geog_results/results_kn_'+scn+'_D.csv', D, delimiter=',')

## Analysis

In [ ]:
results_dict_norm_to_bl = {}
for scn in scenarios:
    print(scn)
    results_dict_norm_to_bl[scn] = {}
    
    results_dict_norm_to_bl[scn]['E'] = np.genfromtxt('data/geog_results/results_omng_'+scn+'_E.csv')
    results_dict_norm_to_bl[scn]['U'] = np.genfromtxt('data/geog_results/results_omng_'+scn+'_U.csv')
    results_dict_norm_to_bl[scn]['V'] = np.genfromtxt('data/geog_results/results_omng_'+scn+'_V.csv')
    results_dict_norm_to_bl[scn]['V_lt'] = np.genfromtxt('data/geog_results/results_omng_'+scn+'_V_lt.csv')
    results_dict_norm_to_bl[scn]['D'] = np.genfromtxt('data/geog_results/results_omng_'+scn+'_D.csv')
    


In [ ]:
results_dict_norm_kn_to_bl = {}
for scn in scenarios:
    print(scn)
    results_dict_norm_kn_to_bl[scn] = {}
    
    results_dict_norm_kn_to_bl[scn]['E'] = np.genfromtxt('data/geog_results/results_kn_'+scn+'_E.csv')
    results_dict_norm_kn_to_bl[scn]['U'] = np.genfromtxt('data/geog_results/results_kn_'+scn+'_U.csv')
    results_dict_norm_kn_to_bl[scn]['V'] = np.genfromtxt('data/geog_results/results_kn_'+scn+'_V.csv')
    results_dict_norm_kn_to_bl[scn]['V_lt'] = np.genfromtxt('data/geog_results/results_kn_'+scn+'_V_lt.csv')
    results_dict_norm_kn_to_bl[scn]['D'] = np.genfromtxt('data/geog_results/results_kn_'+scn+'_D.csv')
    


In [ ]:
scn = scenarios[1:2][0]
x_data_0 = 100*(np.array((scenario_dict_norm_to_bl[scn].loc[2030] - scenario_dict_norm_to_bl['Cen2BAseq'].loc[2030]).values
             )/scenario_dict_norm_to_bl['Cen2BAseq'].loc[2030])


y_data_0 = percentage_POINT_change_u_to_baseline(results_dict_norm_to_bl[scn]['E'], 
                                           results_dict_norm_to_bl[scn]['U'],
                                           results_dict_norm_to_bl['Cen2BAseq']['E'], 
                                           results_dict_norm_to_bl['Cen2BAseq']['U'],
                                            260,416)

y_data_kn_0 = percentage_POINT_change_u_to_baseline(results_dict_norm_kn_to_bl[scn]['E'], 
                                           results_dict_norm_kn_to_bl[scn]['U'],
                                           results_dict_norm_kn_to_bl['Cen2BAseq']['E'], 
                                           results_dict_norm_kn_to_bl['Cen2BAseq']['U'],
                                            260,416)

scn = scenarios[2:3][0]
x_data = 100*(np.array((scenario_dict_norm_to_bl[scn].loc[2030] - scenario_dict_norm_to_bl['Cen2BAseq'].loc[2030]).values
             )/scenario_dict_norm_to_bl['Cen2BAseq'].loc[2030])

y_data = percentage_POINT_change_u_to_baseline(results_dict_norm_to_bl[scn]['E'], 
                                           results_dict_norm_to_bl[scn]['U'],
                                           results_dict_norm_to_bl['Cen2BAseq']['E'], 
                                           results_dict_norm_to_bl['Cen2BAseq']['U'],
                                            260,416)

y_data_kn = percentage_POINT_change_u_to_baseline(results_dict_norm_kn_to_bl[scn]['E'], 
                                           results_dict_norm_kn_to_bl[scn]['U'],
                                           results_dict_norm_kn_to_bl['Cen2BAseq']['E'], 
                                           results_dict_norm_kn_to_bl['Cen2BAseq']['U'],
                                            260,416)



In [ ]:
results_df = pd.DataFrame(x_data.copy()).rename(columns={2030: 'x_data'})
results_df.reset_index(inplace=True)
results_df.rename(columns={'index': 'occ-loc'}, inplace=True)
results_df['occ_code'] = [occ[:4] for occ in results_df['occ-loc']]
results_df['loc'] = [occ[4:] for occ in results_df['occ-loc']]
results_df['emp_0'] = employment_0

results_df['occ_1'] = [int(occ[0]) for occ in results_df['occ_code']]
results_df['occ_2'] = [int(occ[:2]) if not occ[0]=='0' else occ[0] for occ in results_df['occ_code']]
results_df['loc_code'] = [np.where(np.array(regions)==reg)[0][0] for reg in results_df['loc']]
results_df.head()

## Labelled unemployment figures

In [ ]:
cbo_1 = pd.read_csv(data_path+'CBO2002_occ_titles_1digit.csv')
cbo_1.loc[0,'TITLE'] = 'Managers'
xlabels_1digit = [re.sub("(.{"+str(i)+"})", "\\1\n", label, 1, re.DOTALL) for i, label in 
           zip([30,25,30,30,27,27,21,21,22], list(cbo_1['TITLE']))]

In [ ]:
reg_cols = {'RSul': (244,158,196), #Lavender
 'Parana': (121,37,0), #Brown
'MtGrSul': (237,1,125), #RubineRed
'MtGrosso': (0,174,239), #Cyan
'Rondonia': (0,166,79), #Green
'Acre': (238,41,103), #WildStrawberry
'GoiasDF': (0,128,128), #teal
'Roraima': (218,157,118), #Tan
'Amapa': (60,128,49), #OliveGreen
'Amazonas': (0,181,190), #Aquamarine
'Para': (146,38,143), #Plum
'Matopiba': (223,230,116), #GreenYellow
'RNordeste': (141,199,62), #LimeGreen
'Bahia': (150,75,0), #brown
'RSudeste': (70,197,221), #SkyBlue
 'SaoPaulo': (255,128,0),} #orange

In [ ]:
results_df_large_occs_only = results_df.iloc[large_occs].copy()

In [ ]:
results_df['loc_col'] = [np.array(reg_cols[reg])/255 for reg in results_df['loc']]
results_df_large_occs_only['loc_col'] = [np.array(reg_cols[reg])/255 for reg in results_df_large_occs_only['loc']]

In [ ]:
results_df['occ_title'] = [cbo_occ_titles_merge1[cbo_occ_titles_merge1['CODE']==oc]['TITLE'].values[0] 
                           for oc in results_df['occ_code']]

In [ ]:
results_df['full_name'] = results_df['occ_title'] + ' in ' + results_df['loc'] 

In [ ]:
reg_label_dict = {'Acre':'Acre',
 'Amapa':'Amapá',
 'Amazonas':'Amazonas',
 'Bahia':'Bahia',
 'GoiasDF':'GoiásDF',
 'Matopiba':'Matopiba',
 'MtGrSul':'MtGrSul',
 'MtGrosso':'MtGrosso',
 'Para':'Pará',
 'Parana':'Paraná',
 'RNordeste':'RNordeste',
 'RSudeste':'RSudeste',
 'RSul':'RSul',
 'Rondonia':'Rondônia',
 'Roraima':'Roraima',
 'SaoPaulo':'SãoPaulo'}

In [ ]:
short_dict_agr = {
'Garment sewing machine operators in RSul': 'Clothing machinists',
'Store and market trade operators in SaoPaulo': 'Store clerks',
'Agricultural workers in vegetable cultivation in SaoPaulo': 'Agricultural\nworkers\n(vegetable)',
'Textile production inspectors and proofreaders in RSul': 'Textile\ninspectors',
'Elementary school college level teachers (first to fourth grade) in RSudeste': 
                                                'Early elementary teachers',
'Secondary level teachers in elementary school in RSudeste': 'Elementary teachers',
'Secondary level teachers in elementary school in SaoPaulo': 'Elementary teachers',
'Secondary level teachers in elementary school in RSul': 'Elementary teachers',
'Masonry structural workers in MtGrosso': 'Masonry\nstructural\nworkers',
'Civil construction helpers in MtGrosso': 'Civil\nconstruction\nhelpers',
'Civil construction helpers in Amazonas': 'Civil\nconstruction\nhelpers',
'Masonry structural workers in RSudeste': 'Masonry\nstructural\nworkers',
'Civil construction helpers in Matopiba': 'Civil construction\nhelpers',
'Masonry structural workers in RSul': 'Masonry structural\nworkers',
'Agricultural support workers in RSudeste': 'Agricultural support workers',
'Cashiers and ticket clerks (except bank tellers) in RSudeste': 'Cashiers ',
'Production line feeders in SaoPaulo': 'Production line\nfeeders',
'Agricultural workers in vegetable cultivation in Bahia': 'Agricultural\nworkers\n(vegetable)',
'Machine and lifting equipment operators in RSudeste': 'Machine\noperators',
'Assemblers of machines, equipment and accessories on assembly lines in RSul': 'Assemblers',
'Garment sewing machine operators in RSudeste': 'Clothing machinists',
'Building maintenance workers in SaoPaulo': 'Building\nmaintenance\nworkers',
'Urban, metropolitan and road bus drivers in RSudeste': 'Bus\ndrivers',
'Grass cultivation agricultural labourers and Growing of fibrous crops agricultural labourers and Olericulture agricultural labourers and Flower and ornamental plant cultivation agricultural labourers and Fruit cultivation agricultural labourers and Agricultural workers in vegetable cultivation and Agricultural workers in oil-seed crops and Agricultural workers in spice, aromatic and medicinal crops in Roraima': 'Agricultural\nlabourers',
'Medium livestock keepers and Poultry and rabbit farmers and Insect and companion animal breeders in RSul': 'Medium livestock keepers',
'Construction site managers in a construction company in RSul': 'Construction\nsite managers',
'Civil & Allied Engineers in MtGrSul': 'Civil and Allied\nEngineers',
'Agricultural workers in general in RSul': 'General agricultural workers',
'Agricultural mechanization workers in Roraima': 'Agricultural\nmechanization\nworkers',
'Agricultural workers in general in Rondonia': 'General\nagricultural\nworkers',
}

In [ ]:
save = True

In [ ]:
agr_unemp_occs = [3912, 4052, 2416, 4071, 4100, 362, 657, 4037, 4182, 4038, 2413, 2412]

In [ ]:
results_df['x_agr'] = x_data_0.values
results_df['x_manuf'] = x_data.values

results_df['y_agr'] = y_data_0
results_df['y_manuf'] = y_data

In [ ]:
f, axs = plt.subplots(1,1, figsize=(9, 7))
# f.tight_layout(pad=8.0)

# f.subplots_adjust(right=0.75)


large_occs = list(np.where(employment_0 >= 1000)[0])
print(len(large_occs))
size_emp = np.array([35 + 0.0001*(employment_0[i]) for i in range(len(employment_0))])

axs.spines['top'].set_visible(False)
axs.spines['right'].set_visible(False)

m = ["v","H",">", "o", "D", "p", "d", "h", "<", (7,0), "*", "X", "s", "^", "8", "P"] 
    
scn = scenarios[1:2][0]


axs.scatter((x_data_0.iloc[large_occs]), (y_data_kn_0[large_occs]), alpha=0.6, #label='Kn
            zorder=3, s=4)#marker='s')

for i, reg in enumerate(regions):
    large_occs = list(np.where((employment_0 >= 1000) & (results_df['loc']==reg))[0])

    if reg not in ['RSudeste', 'SaoPaulo', 'RSul', 'MtGrSul', 'Rondonia', 'Roraima']:
        axs.scatter((x_data_0.iloc[large_occs]), (y_data_0[large_occs]), s=size_emp[large_occs],
                         alpha=0.5, edgecolors='k', zorder=2,  linewidth=0.5,#cmap=plt.get_cmap('tab10', 9),
    #                        vmin=0.5, vmax=9.5,
                           marker=m[i],
                         c='0.8', label=reg_label_dict[reg]
                          )
    else:
        axs.scatter((x_data_0.iloc[large_occs]), (y_data_0[large_occs]), s=size_emp[large_occs],
                         alpha=0.5, edgecolors='k', zorder=2,  linewidth=0.5,#cmap=plt.get_cmap('tab10', 9),
    #                        vmin=0.5, vmax=9.5,
                           marker=m[i],
                         c='0.8', #label=reg
                          )
    
    labelled_occs = [i for i in agr_unemp_occs if i in list(results_df[results_df['loc']==reg].index)]
    if len(labelled_occs) !=0 :
        uplot = axs.scatter((results_df['x_agr'].loc[labelled_occs]), 
                            (results_df['y_agr'].loc[labelled_occs]), 
                            s=size_emp[labelled_occs],
                     alpha=1, edgecolors='k', zorder=4, #cmap='inferno', vmin=0.5, vmax=9.5,
                     c=results_df['loc_col'][labelled_occs], marker=m[i], label=reg_label_dict[reg]
                      )

        
xlim = list(axs.get_xlim())
axs.plot(xlim, [0, 0], '--', c='grey', zorder=1)
axs.set_xlabel('2030 Demand change from baseline (%)', fontsize=20)
axs.set_ylabel('2018-2030 Average unemployment \nrate change from baseline (%-point)', fontsize=20)
axs.set_xlim(xlim)
# axs.set_title('Agriculture growth path', fontsize=20)

    
import matplotlib.lines as mlines

kn_marker = mlines.Line2D([], [], color='tab:blue', marker='o', linestyle='None', markeredgewidth=0,
                          markersize=10, label='K$_n$')
omn_marker = mlines.Line2D([], [], color='0.8', marker='s', linestyle='None', markeredgewidth=0.2,
                          markersize=10, markeredgecolor='black', label='OMN')

legend2 = axs.legend(handles=[kn_marker, omn_marker], loc='lower left', fontsize=14)
axs.add_artist(legend2)



legend = axs.legend(title='Region', loc=(1.04,0.1), fontsize='12')
legend.get_title().set_fontsize('14')
for i in range(len(regions)):
    legend.legendHandles[i]._sizes = [100]

# axs.add_artist(legend)
    
short_dict = short_dict_agr
    
for j, occ_code in enumerate(agr_unemp_occs[:10]):
    print(j, short_dict[results_df['full_name'].iloc[occ_code]])
    ha = 'left'
    va = 'center'
    x_alt = 2
    y_alt = 0.02
    if (j<2)or(j==7):
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data_0.iloc[agr_unemp_occs].values[j]+x_alt, y_data_0[agr_unemp_occs][j]+y_alt), 
#                     xytext=(x_data_0[agr_unemp_occs][j]+x_alt, y_data_0[agr_unemp_occs][j]+y_alt),
#                     arrowprops=dict(arrowstyle= '->', color='black', lw=1),
                    va=va, ha=ha, fontsize=11, c='black', )#fontweight='bold')  
    elif (j==6):
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data_0.iloc[agr_unemp_occs].values[j]+0.5, y_data_0[agr_unemp_occs][j]), 
                    xytext=(x_data_0.iloc[agr_unemp_occs].values[j]+8, y_data_0[agr_unemp_occs][j]+0.2),
                    arrowprops=dict(arrowstyle= '->', color='black', lw=1),
                    va=va, ha=ha, fontsize=11, c='black', )#fontweight='bold')  
    elif (j==3):
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data_0.iloc[agr_unemp_occs].values[j], y_data_0[agr_unemp_occs][j]+0.01), 
                    xytext=(x_data_0.iloc[agr_unemp_occs].values[j]-7, y_data_0[agr_unemp_occs][j]+0.178),
                    arrowprops=dict(arrowstyle= '->', color='black', lw=1),
                    va=va, ha=ha, fontsize=11, c='black', )#fontweight='bold')  
    elif (j==4):
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data_0.iloc[agr_unemp_occs].values[j]+0.5, y_data_0[agr_unemp_occs][j]+y_alt), 
                    xytext=(x_data_0.iloc[agr_unemp_occs].values[j]+6, y_data_0[agr_unemp_occs][j]+0.3),
                    arrowprops=dict(arrowstyle= '->', color='black', lw=1, relpos=(0.1,0)),
                    va=va, ha=ha, fontsize=11, c='black', )#fontweight='bold')  
    elif (j==5):
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data_0.iloc[agr_unemp_occs].values[j]+0.5, y_data_0[agr_unemp_occs][j]-y_alt), 
                    xytext=(x_data_0.iloc[agr_unemp_occs].values[j]+6, y_data_0[agr_unemp_occs][j]-0.3),
                    arrowprops=dict(arrowstyle= '->', color='black', lw=1, relpos=(0.1,0)),
                    va=va, ha=ha, fontsize=11, c='black', )#fontweight='bold')  
    elif (j==8):
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data_0.iloc[agr_unemp_occs].values[j], y_data_0[agr_unemp_occs][j]-y_alt), 
                    xytext=(x_data_0.iloc[agr_unemp_occs].values[j]-7, y_data_0[agr_unemp_occs][j]-0.8),
                    arrowprops=dict(arrowstyle= '->', color='black', lw=1, relpos=(0.1,0)),
                    va=va, ha=ha, fontsize=11, c='black', )#fontweight='bold')  
    elif (j==9):
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data_0.iloc[agr_unemp_occs].values[j], y_data_0[agr_unemp_occs][j]-y_alt), 
                    xytext=(x_data_0.iloc[agr_unemp_occs].values[j]-8, y_data_0[agr_unemp_occs][j]-0.35),
                    arrowprops=dict(arrowstyle= '->', color='black', lw=1, relpos=(0.1,0)),
                    va=va, ha=ha, fontsize=11, c='black', )#fontweight='bold')      
    else:
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data_0.iloc[agr_unemp_occs].values[j]+0.5, y_data_0[agr_unemp_occs][j]-y_alt), 
                    xytext=(x_data_0.iloc[agr_unemp_occs].values[j]+6, y_data_0[agr_unemp_occs][j]-7*y_alt),
                    arrowprops=dict(arrowstyle= '->', color='black', lw=1, relpos=(0,1)),
                    va=va, ha=ha, fontsize=11, c='black', )#fontweight='bold')  

        
if save == True:
    plt.savefig('figures/unemp_agr.pdf',bbox_inches='tight')
    plt.savefig('figures/unemp_agr.png',bbox_inches='tight')    


# plt.show()

In [ ]:
short_dict_manuf = {
'Garment sewing machine operators in RSul': 'Clothing machinists',
'General-purpose footwear manufacturing workers in Bahia': 'Footwear\nmanufacturing\nworkers',
'Shoe sewing and fitting machine operators in RSudeste': 'Shoe\nmachine\noperators',
'Equipment operators in the production of bread, pasta, sweets, chocolates and chocolate products in SaoPaulo': 'Bakery\nequipment\noperators',
'Textile workers in SaoPaulo': 'Textile\nworkers',
'Garment sewing machine operators in SaoPaulo': 'Clothing\nmachinists',
'Crystal, glass, ceramic, porcelain, fiberglass, abrasive and related manufacturing and processing equipment operators in SaoPaulo': 'Glass\nequipment\noperators',
'Grass cultivation agricultural labourers in Matopiba': 'Agricultural\nworkers (grass)',
'Electro-electronic equipment assemblers in SaoPaulo': 'Electronic\nequipment\nassemblers',
'Electromechanical installers and maintainers of elevators, stairways and automatic doors in RSul': 'Electromechanical installers',
'Civil construction helpers in Amazonas': 'Civil construction\nhelpers',
'Masonry structural workers in RSudeste': 'Masonry structural\nworkers',
'Civil construction helpers in MtGrosso': 'Civil construction\nhelpers',
'Masonry structural workers in MtGrosso': 'Masonry structural\nworkers',
'Administrative agents, assistants and helpers in RSudeste': 'Administrative\nagents',
'Delivery motorcyclists and delivery cyclists in RSudeste': 'Delivery\ncyclists',
}

In [ ]:
manuf_unemp_occs = []
for k in short_dict_manuf.keys():
    if k in results_df['full_name'].values:
        manuf_unemp_occs.append(results_df[results_df['full_name']==k].index[0])
manuf_unemp_occs.append(results_df[results_df['occ-loc']=='761XSaoPaulo'].index[0])
print(manuf_unemp_occs)

In [ ]:
short_dict_manuf['Textile workers and Textile fiber sorting and wool washing workers and Spinning mill operators and Weaving machine and similar machine operators in SaoPaulo'] = 'Textile\nworkers'

In [ ]:
manuf_unemp_occs = [5037,5101, 4502, 5891, 4895, 6380,  4335, 4507, 4331, 3124, 3828]

In [ ]:
save = True
# save = False

In [ ]:
f, axs = plt.subplots(1,1, figsize=(9, 7))
# f.tight_layout(pad=8.0)
large_occs = list(np.where(employment_0 >= 1000)[0])
print(len(large_occs))
size_emp = np.array([35 + 0.0001*(employment_0[i]) for i in range(len(employment_0))])

axs.spines['top'].set_visible(False)
axs.spines['right'].set_visible(False)

m = ["v","H",">", "o", "D", "p", "d", "h", "<", (7,0), "*", "X", "s", "^", "8", "P"] 
    
scn = scenarios[1:2][0]

axs.scatter((x_data.iloc[large_occs]), (y_data_kn[large_occs]), alpha=0.6, #label='Kn
            zorder=3, s=4)#marker='s')

for i, reg in enumerate(regions):
    large_occs = list(np.where((employment_0 >= 1000) & (results_df['loc']==reg))[0])

    if reg not in ['SaoPaulo', 'Amazonas', 'MtGrosso', 'RSudeste', 'RSul']:
        axs.scatter((x_data.iloc[large_occs]), (y_data[large_occs]), s=size_emp[large_occs],
                         alpha=0.5, edgecolors='k', zorder=2,  linewidth=0.5,#cmap=plt.get_cmap('tab10', 9),
    #                        vmin=0.5, vmax=9.5,
                           marker=m[i],
                         c='0.8', label=reg_label_dict[reg]
                          )
    else:
        axs.scatter((x_data.iloc[large_occs]), (y_data[large_occs]), s=size_emp[large_occs],
                         alpha=0.5, edgecolors='k', zorder=2,  linewidth=0.5,#cmap=plt.get_cmap('tab10', 9),
    #                        vmin=0.5, vmax=9.5,
                           marker=m[i],
                         c='0.8', #label=reg
                          )
    
    labelled_occs = [i for i in manuf_unemp_occs if i in list(results_df[results_df['loc']==reg].index)]
    if len(labelled_occs) != 0:
        uplot = axs.scatter((results_df['x_manuf'].loc[labelled_occs]), 
                            (results_df['y_manuf'].loc[labelled_occs]), 
                            s=size_emp[labelled_occs],
                     alpha=1, edgecolors='k', zorder=4, #cmap='inferno', vmin=0.5, vmax=9.5,
                     c=results_df['loc_col'][labelled_occs], marker=m[i], label=reg_label_dict[reg]
                      )

xlim = list(axs.get_xlim())
axs.plot(xlim, [0, 0], '--', c='grey', zorder=1)
axs.set_xlabel('2030 Demand change from baseline (%)', fontsize=20)
axs.set_ylabel('2018-2030 Average unemployment \nrate change from baseline (%-point)', fontsize=20)
axs.set_xlim(xlim)
# axs.set_title('Agriculture growth path', fontsize=20)

import matplotlib.lines as mlines

kn_marker = mlines.Line2D([], [], color='tab:blue', marker='o', linestyle='None', markeredgewidth=0,
                          markersize=10, label='K$_n$')
omn_marker = mlines.Line2D([], [], color='0.8', marker='s', linestyle='None', markeredgewidth=0.2,
                          markersize=10, markeredgecolor='black', label='OMN')

legend2 = axs.legend(handles=[kn_marker, omn_marker], loc='lower left', fontsize=14)
axs.add_artist(legend2)



legend = axs.legend(title='Region', loc=(1.04,0.1), fontsize='12')
legend.get_title().set_fontsize('14')
for i in range(len(regions)):
    legend.legendHandles[i]._sizes = [100]
    
short_dict = short_dict_manuf
    
for j, occ_code in enumerate(manuf_unemp_occs):
    print(j, short_dict[results_df['full_name'].iloc[occ_code]])
    ha = 'center'
    va = 'center'
    x_alt = 1
    y_alt = 0.02
    if j==0:
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data.iloc[manuf_unemp_occs].values[j], y_data[manuf_unemp_occs][j]-0.006), 
                    xytext=(x_data.iloc[manuf_unemp_occs].values[j]+x_alt, y_data[manuf_unemp_occs][j]-5*y_alt),
                    arrowprops=dict(arrowstyle= '->', color='black', lw=1),
                    va=va, ha=ha, fontsize=10, c='black', )#fontweight='bold')  
    elif j==3:
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data.iloc[manuf_unemp_occs].values[j]-x_alt, y_data[manuf_unemp_occs][j]-y_alt),
                    va='top', ha=ha, fontsize=10, c='black', )#fontweight='bold')   
    elif j==4:
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data.iloc[manuf_unemp_occs].values[j]-0.5, y_data[manuf_unemp_occs][j]+y_alt/2), 
                    va='center', ha='right', fontsize=10, c='black', )#fontweight='bold')   
    elif j==5:
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data.iloc[manuf_unemp_occs].values[j]+2, y_data[manuf_unemp_occs][j]-y_alt), 
                    va='top', ha='right', fontsize=10, c='black', )#fontweight='bold') 
    elif j==7:
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data.iloc[manuf_unemp_occs].values[j], y_data[manuf_unemp_occs][j]-0.006), 
                    xytext=(x_data.iloc[manuf_unemp_occs].values[j]+x_alt, y_data[manuf_unemp_occs][j]-5*y_alt),
                    arrowprops=dict(arrowstyle= '->', color='black', lw=1),
                    va=va, ha=ha, fontsize=10, c='black', )#fontweight='bold')  
    elif j==8:
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data.iloc[manuf_unemp_occs].values[j], y_data[manuf_unemp_occs][j]+0.006), 
                    xytext=(x_data.iloc[manuf_unemp_occs].values[j]+x_alt, y_data[manuf_unemp_occs][j]+5*y_alt),
                    arrowprops=dict(arrowstyle= '->', color='black', lw=1),
                    va=va, ha=ha, fontsize=10, c='black', )#fontweight='bold')  
    elif j==9:
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data.iloc[manuf_unemp_occs].values[j], y_data[manuf_unemp_occs][j]+0.006), 
                    xytext=(x_data.iloc[manuf_unemp_occs].values[j]+2.5, y_data[manuf_unemp_occs][j]+10*y_alt),
                    arrowprops=dict(arrowstyle= '->', color='black', lw=1, relpos=(0.35, 0)),
                    va=va, ha=ha, fontsize=10, c='black', )#fontweight='bold')  
    elif j==10:
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data.iloc[manuf_unemp_occs].values[j], y_data[manuf_unemp_occs][j]+0.006), 
                    xytext=(x_data.iloc[manuf_unemp_occs].values[j]-x_alt, y_data[manuf_unemp_occs][j]+10*y_alt),
                    arrowprops=dict(arrowstyle= '->', color='black', lw=1, relpos=(0.8,0)),
                    va=va, ha=ha, fontsize=10, c='black', )#fontweight='bold')  
    else:
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data.iloc[manuf_unemp_occs].values[j]+x_alt, y_data[manuf_unemp_occs][j]+2*y_alt), 
#                     xytext=(x_data.iloc[manuf_unemp_occs].values[j]+x_alt, y_data[manuf_unemp_occs][j]+y_alt),
#                     arrowprops=dict(arrowstyle= '->', color='black', lw=1),
                    va=va, ha=ha, fontsize=10, c='black', )#fontweight='bold')  


if save == True:
    plt.savefig('figures/unemp_manuf.pdf',bbox_inches='tight')
    plt.savefig('figures/unemp_manuf.png',bbox_inches='tight')    


# plt.show()

## Vacancies

In [ ]:
def period_vac_rate(D, V, time_start, time_end):
    d = D[time_start:time_end, :]
    v = V[time_start:time_end, :]
    return 100*sum(v) / sum(d)

def percentage_POINT_change_v_to_baseline(D, U, D_b, U_b, time_start_1, time_end_1):
    v_scn_num = period_vac_rate(D, U, time_start_1, time_end_1)
    v_baseline_num = period_vac_rate(D_b, U_b, time_start_1, time_end_1)
    return v_scn_num - v_baseline_num


In [ ]:
scn = scenarios[1:2][0]

vacs_0 = percentage_POINT_change_v_to_baseline(results_dict_norm_to_bl[scn]['V'] +
                                                  results_dict_norm_to_bl[scn]['E'], 
                                                  results_dict_norm_to_bl[scn]['V_lt'],
                                                  results_dict_norm_to_bl['Cen2BAseq']['V'] +
                                                  results_dict_norm_to_bl['Cen2BAseq']['E'], 
                                                  results_dict_norm_to_bl['Cen2BAseq']['V_lt'],
                                                  260,416)

vacs_kn_0 = percentage_POINT_change_v_to_baseline(results_dict_norm_kn_to_bl[scn]['V'] +
                                                  results_dict_norm_kn_to_bl[scn]['E'], 
                                                  results_dict_norm_kn_to_bl[scn]['V_lt'],
                                                  results_dict_norm_kn_to_bl['Cen2BAseq']['V'] +
                                                  results_dict_norm_kn_to_bl['Cen2BAseq']['E'], 
                                                  results_dict_norm_kn_to_bl['Cen2BAseq']['V_lt'],
                                                  260,416)


scn = scenarios[2:3][0]


vacs = percentage_POINT_change_v_to_baseline(results_dict_norm_to_bl[scn]['V'] +
                                                  results_dict_norm_to_bl[scn]['E'], 
                                                  results_dict_norm_to_bl[scn]['V_lt'],
                                                  results_dict_norm_to_bl['Cen2BAseq']['V'] +
                                                  results_dict_norm_to_bl['Cen2BAseq']['E'], 
                                                  results_dict_norm_to_bl['Cen2BAseq']['V_lt'],
                                                  260,416)

vacs_kn = percentage_POINT_change_v_to_baseline(results_dict_norm_kn_to_bl[scn]['V'] +
                                                  results_dict_norm_kn_to_bl[scn]['E'], 
                                                  results_dict_norm_kn_to_bl[scn]['V_lt'],
                                                  results_dict_norm_kn_to_bl['Cen2BAseq']['V'] +
                                                  results_dict_norm_kn_to_bl['Cen2BAseq']['E'], 
                                                  results_dict_norm_kn_to_bl['Cen2BAseq']['V_lt'],
                                                  260,416)


## Labelled figures

In [ ]:
short_dict_agr = {    'Public prosecutors and attorneys in GoiasDF': 'Public prosecutors',
'Higher level teachers in early childhood education in Bahia': 'Early childhood teachers',
'Electric, telephone and data communication lines and cables installers and repairers in MtGrosso': 
    'Cable installers\nand repairers',
'Systems and applications development technicians in Rondonia': 
    'Systems technicians',
'Programmers, evaluators, and guidance counselors in Amazonas': 
    'Programmers, evaluators, and\nguidance counselors',
'Electric, telephone and data communication lines and cables installers and repairers in Para': 
    'Cable installers\nand repairers',
'Civil construction helpers in MtGrosso': 'Civil construction\nhelpers',
'Masonry structural workers in MtGrosso': 'Masonry structural\nworkers',
'Clinical Doctors in Para': 'Clinical Doctors',
'Programmers, evaluators, and guidance counselors in Para': 
    'Programmers, evaluators, and\nguidance counselors',
'Weaving machine and similar machine operators in RNordeste': 'Weaving machinists ',
'Spinning mill operators in RSul': 'Spinning mill operators',
'Weaving machine and similar machine operators in RSudeste': 'Weaving machinists',
'Pharmaceutical, cosmetic and related products machine and plant operators in GoiasDF': 
    'Pharmaceutical plant operators',
'Large animal husbandry workers in Rondonia': 'Large animal husbandry workers',
'Weaving machine and similar machine operators in RSul': 
    'Weaving\nmachinists',
'Planning, scheduling, and logistics control professionals in SaoPaulo': 
    'Logistics control professionals',
    'Systems and applications development technicians in MtGrosso': 'Systems technicians',
    'Telecommunications Technicians in SaoPaulo': 'Telecommunications\ntechnicians',
    'Agricultural workers in general in Amapa': 'General agricultural workers',
    'Farm supervisors in Rondonia': 'Farm supervisors',
    'Ceramic, tile, stone and wood tile layers in RSul': 'Tile layers',
'Concrete finishers in Matopiba': 'Concrete finishers',
'Subway maintenance mechanic in Parana': 'Subway maintenance\nmechanics',
'Production and operations managers in agriculture, fishing, aquaculture and forestry companies in RSul': 'Agriculture\noperations\nmanagers',
'Medium livestock keepers and Poultry and rabbit farmers and Insect and companion animal breeders in Rondonia': 'Medium livestock keepers',
'Operational permanent way maintenance workers (except railroad) in Roraima': 'Road maintenance workers',
'Agricultural producers in general and Multipurpose agricultural producers and Agricultural Producers in Grass Crops and Growers in fibrous crops and Growers in horticulture and Growers in flower and ornamentals crops and Growers in fruit growing and Agricultural producers in the culture of stimulant plants and Agricultural producers in oil-seed crops and Growers of spices, aromatic and medicinal plants in MtGrSul': 'Agricultural producers',
                'Civil & Allied Engineers in MtGrSul': 'Civil and Allied\nEngineers'
}

In [ ]:
agr_vacs_occs = []
for k in short_dict_agr.keys():
    if k in results_df['full_name'].values:
        agr_vacs_occs.append(results_df[results_df['full_name']==k].index[0])
# manuf_vacs_occs.append(results_df[results_df['occ-loc']=='761XSaoPaulo'].index[0])
print(agr_vacs_occs)

In [ ]:
# agr_vacs_occs = [2017, 2111]
# agr_vacs_occs = [2017, 2111, 1262, 973, 4919, 4920, 4507, 4331, 946, 5629, 2105]
agr_vacs_occs = [4026, 2017, 2111, 4464, 4425, 6201, 330, 4101, 6494, 3983, 657, 4920]

In [ ]:
save

In [ ]:
f, axs = plt.subplots(1,1, figsize=(9, 7))
# f.tight_layout(pad=8.0)
large_occs = list(np.where(employment_0 >= 1000)[0])
#     large_occs = list(range(n))
print(len(large_occs))
size_emp = np.array([35 + 0.0001*(employment_0[i]) for i in range(len(employment_0))])

axs.spines['top'].set_visible(False)
axs.spines['right'].set_visible(False)

m = ["v","H",">", "o", "D", "p", "d", "h", "<", (7,0), "*", "X", "s", "^", "8", "P"] 
    
scn = scenarios[1:2][0]
axs.scatter((x_data_0.iloc[large_occs]), (vacs_kn_0[large_occs]), alpha=0.6, #label='Kn
            zorder=3, s=4)#marker='s')

for i, reg in enumerate(regions):
    large_occs = list(np.where((employment_0 >= 1000) & (results_df['loc']==reg))[0])
#     print(len(large_occs))
    if reg not in ['Rondonia', 'SaoPaulo', 'Amapa', 'Roraima', 'Parana', 'RSul', 'MtGrSul', 'Matopiba',
                  'Para']:
 
        axs.scatter((x_data_0.iloc[large_occs]), (vacs_0[large_occs]), s=size_emp[large_occs],
                     alpha=0.5, edgecolors='k', zorder=2, linewidth=0.5,#cmap=plt.get_cmap('tab10', 9),
#                        vmin=0.5, vmax=9.5,
                       marker=m[i],
                     c='0.8', label=reg_label_dict[reg]
                      )
    else:
        axs.scatter((x_data_0.iloc[large_occs]), (vacs_0[large_occs]), s=size_emp[large_occs],
                     alpha=0.5, edgecolors='k', zorder=2,  linewidth=0.5,#cmap=plt.get_cmap('tab10', 9),
#                        vmin=0.5, vmax=9.5,
                       marker=m[i],
                     c='0.8', #label=reg
                      )
    labelled_occs = [i for i in agr_vacs_occs if i in list(results_df[results_df['loc']==reg].index)]
    if len(labelled_occs) != 0:
        uplot = axs.scatter((x_data_0.iloc[labelled_occs]), (vacs_0[labelled_occs]), s=size_emp[labelled_occs],
                     alpha=1, edgecolors='k', zorder=4, #cmap='inferno', vmin=0.5, vmax=9.5,
                     c=results_df['loc_col'][labelled_occs], marker=m[i], label=reg_label_dict[reg]
                      )

xlim = list(axs.get_xlim())
axs.plot(xlim, [0, 0], '--', c='grey', zorder=1)
axs.set_xlabel('2030 Demand change from baseline (%)', fontsize=20)
axs.set_ylabel('2018-2030 Average 6-month\n unfilled vacancy rate change \nfrom baseline (%-point)', fontsize=20)
axs.set_xlim(xlim)
# axs.set_title('Higher-emissions Scenario', fontsize=20)
axs.tick_params(labelsize=14)


import matplotlib.lines as mlines

kn_marker = mlines.Line2D([], [], color='tab:blue', marker='o', linestyle='None', markeredgewidth=0,
                          markersize=10, label='K$_n$')
omn_marker = mlines.Line2D([], [], color='0.8', marker='s', linestyle='None', markeredgewidth=0.2,
                          markersize=10, markeredgecolor='black', label='OMN')

legend2 = axs.legend(handles=[kn_marker, omn_marker], loc='lower right', fontsize=14)
axs.add_artist(legend2)

legend = axs.legend(title='Region', loc=(1.04,0.1), fontsize='12')
legend.get_title().set_fontsize('14')
for i in range(len(regions)):
    legend.legendHandles[i]._sizes = [100]
    
    
short_dict = short_dict_agr
    
for j, occ_code in enumerate(agr_vacs_occs):
    print(j, short_dict[results_df['full_name'].iloc[occ_code]])
    ha = 'center'
    va = 'center'
    x_alt = 1
    y_alt = 0.02
    if (j<=1)or(j==7)or(j>=9):
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data_0.iloc[agr_vacs_occs].values[j]+x_alt, vacs_0[agr_vacs_occs][j]-2*y_alt), 
#                     xytext=(x_data_0[agr_vacs_occs][j]+x_alt, vacs_0[agr_vacs_occs][j]+y_alt),
#                     arrowprops=dict(arrowstyle= '->', color='black', lw=1),
                    va='top', ha='left', fontsize=10, c='black', )#fontweight='bold')  
    elif (j==2)or(j==4):
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data_0.iloc[agr_vacs_occs].values[j], vacs_0[agr_vacs_occs][j]+y_alt), 
                    xytext=(x_data_0.iloc[agr_vacs_occs].values[j]-x_alt, vacs_0[agr_vacs_occs][j]+0.5),
                    arrowprops=dict(arrowstyle= '->', color='black', lw=1, relpos=(0.8, 0)),
                    va=va, ha='right', fontsize=10, c='black', )#fontweight='bold') 
    elif j==3:
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data_0.iloc[agr_vacs_occs].values[j]+1, vacs_0[agr_vacs_occs][j]+y_alt), 
                    xytext=(x_data_0.iloc[agr_vacs_occs].values[j]+12, vacs_0[agr_vacs_occs][j]+0.385),
                    arrowprops=dict(arrowstyle= '->', color='black', lw=1, ),#relpos=(0.2, 0)),
                    va=va, ha=ha, fontsize=10, c='black', )#fontweight='bold') 
    elif (j==5)or(j==8):
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data_0.iloc[agr_vacs_occs].values[j], vacs_0[agr_vacs_occs][j]+y_alt), 
                    xytext=(x_data_0.iloc[agr_vacs_occs].values[j]-1.3*x_alt, vacs_0[agr_vacs_occs][j]+0.5),
                    arrowprops=dict(arrowstyle= '->', color='black', lw=1, relpos=(0.9, 0)),
                    va=va, ha='right', fontsize=10, c='black', )#fontweight='bold') 
    elif j==6:
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data_0.iloc[agr_vacs_occs].values[j]-x_alt, vacs_0[agr_vacs_occs][j]-y_alt), 
                    xytext=(x_data_0.iloc[agr_vacs_occs].values[j]-18, vacs_0[agr_vacs_occs][j]-0.55),
                    arrowprops=dict(arrowstyle= '->', color='black', lw=1, ),#relpos=(0.8, 1)),
                    va=va, ha='left', fontsize=10, c='black', )#fontweight='bold') 
    
    else:
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data_0.iloc[agr_vacs_occs].values[j]+x_alt, vacs_0[agr_vacs_occs][j]+2*y_alt), 
#                     xytext=(x_data_0.iloc[agr_vacs_occs].values[j]+x_alt, vacs_0[agr_vacs_occs][j]+y_alt),
#                     arrowprops=dict(arrowstyle= '->', color='black', lw=1),
                    va=va, ha='right', fontsize=10, c='black', )#fontweight='bold')  

  
if save == True:
    plt.savefig('figures/vacancy_agr.pdf',bbox_inches='tight')
    plt.savefig('figures/vacancy_agr.png',bbox_inches='tight')    


# plt.show()

In [ ]:
short_dict_manuf = {
'Electric, telephone and data communication lines and cables installers and repairers in MtGrosso':
    'Cable installers\nand repairers', 
    'Systems and applications development technicians in Para': 'Systems technicians',
    'Electric, telephone and data communication lines and cables installers and repairers in MtGrosso': 'Cable installers\nand repairers',
'Managers and operations specialists in business, secretarial, and healthcare facilities in Bahia': 'Facilities managers and\noperations specialists',
'Managers and operations specialists in business, secretarial, and healthcare facilities in RNordeste': 'Facilities managers and\noperations specialists',
'Managers of service operations in a tourism, accommodation and food service company in Amazonas': 'Service operations managers',
'Child, youth, adult and elderly caregivers in Amapa': 'Caregivers',
'Multi-skilled garment makers in Bahia': 'Multi-skilled\ngarment\nmakers',
'Garment sewing machine operators in RSul': 'Garment\nsewing\nmachine\noperators',
'Metallurgical furnaces (second melting and reheating) in RSul': 'Metallurgy\nworkers',
'Conventional woodworking machine operators in Parana': 'Woodworking machine operators',
'Cooks in Rondonia': 'Cooks',
'Clothing embroidery and finishing machine operators in Para': 'Clothing embroidery\nmachine operators',
'Mail, parcel and publication sorting and delivery service workers in RSul': 'Mail delivery service workers',
}


In [ ]:
manuf_vacs_occs = []
for k in short_dict_manuf.keys():
    if k in results_df['full_name'].values:
        manuf_vacs_occs.append(results_df[results_df['full_name']==k].index[0])
# manuf_vacs_occs.append(results_df[results_df['occ-loc']=='761XSaoPaulo'].index[0])
print(manuf_vacs_occs)

In [ ]:
manuf_vacs_occs = [4919, 2106, 289, 296, 384, 3658, 5057, 5098, 5763, 5318, 3494, 5110]


In [ ]:
f, axs = plt.subplots(1,1, figsize=(9, 7))
# f.tight_layout(pad=8.0)
large_occs = list(np.where(employment_0 >= 1000)[0])
#     large_occs = list(range(n))
print(len(large_occs))
size_emp = np.array([35 + 0.0001*(employment_0[i]) for i in range(len(employment_0))])

axs.spines['top'].set_visible(False)
axs.spines['right'].set_visible(False)

m = ["v","H",">", "o", "D", "p", "d", "h", "<", (7,0), "*", "X", "s", "^", "8", "P"] 
    
scn = scenarios[1:2][0]
axs.scatter((x_data.iloc[large_occs]), (vacs_kn[large_occs]), alpha=0.6, #label='Kn
            zorder=3, s=4)#marker='s')

for i, reg in enumerate(regions):
    large_occs = list(np.where((employment_0 >= 1000) & (results_df['loc']==reg))[0])
#     print(len(large_occs))
    if reg not in ['Amapa', 'Amazonas', 'Bahia', 'MtGrosso', 'Para', 'Parana', 'RNordeste', 'RSul', 'Rondonia']:
 
        axs.scatter((x_data.iloc[large_occs]), (vacs[large_occs]), s=size_emp[large_occs],
                     alpha=0.5, edgecolors='k', zorder=2, linewidth=0.5,#cmap=plt.get_cmap('tab10', 9),
#                        vmin=0.5, vmax=9.5,
                       marker=m[i],
                     c='0.8', label=reg_label_dict[reg]
                      )
    else:
        axs.scatter((x_data.iloc[large_occs]), (vacs[large_occs]), s=size_emp[large_occs],
                     alpha=0.5, edgecolors='k', zorder=2,  linewidth=0.5,#cmap=plt.get_cmap('tab10', 9),
#                        vmin=0.5, vmax=9.5,
                       marker=m[i],
                     c='0.8', #label=reg
                      )
    labelled_occs = [i for i in manuf_vacs_occs if i in list(results_df[results_df['loc']==reg].index)]
    if len(labelled_occs) != 0:
        uplot = axs.scatter((x_data.iloc[labelled_occs]), (vacs[labelled_occs]), s=size_emp[labelled_occs],
                     alpha=1, edgecolors='k', zorder=4, #cmap='inferno', vmin=0.5, vmax=9.5,
                     c=results_df['loc_col'][labelled_occs], marker=m[i], label=reg_label_dict[reg]
                      )

xlim = list(axs.get_xlim())
axs.plot(xlim, [0, 0], '--', c='grey', zorder=1)
axs.set_xlabel('2030 Demand change from baseline (%)', fontsize=20)
axs.set_ylabel('2018-2030 Average 6-month\n unfilled vacancy rate change \nfrom baseline (%-point)', fontsize=20)
axs.set_xlim(xlim)
# axs.set_title('Higher-emissions Scenario', fontsize=20)
axs.tick_params(labelsize=14)

import matplotlib.lines as mlines

kn_marker = mlines.Line2D([], [], color='tab:blue', marker='o', linestyle='None', markeredgewidth=0,
                          markersize=10, label='K$_n$')
omn_marker = mlines.Line2D([], [], color='0.8', marker='s', linestyle='None', markeredgewidth=0.2,
                          markersize=10, markeredgecolor='black', label='OMN')

legend2 = axs.legend(handles=[kn_marker, omn_marker], loc='lower right', fontsize=14)
axs.add_artist(legend2)

legend = axs.legend(title='Region', loc=(1.04,0.1), fontsize='12')
legend.get_title().set_fontsize('14')
for i in range(len(regions)):
    legend.legendHandles[i]._sizes = [100]
    
    
    
short_dict = short_dict_manuf
    
for j, occ_code in enumerate(manuf_vacs_occs):
    print(j, short_dict[results_df['full_name'].iloc[occ_code]])
    ha = 'center'
    va = 'center'
    x_alt = 0.5
    y_alt = 0.005
    if j==0:
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data.iloc[manuf_vacs_occs].values[j], vacs[manuf_vacs_occs][j]+3*y_alt), 
#                     xytext=(x_data.iloc[manuf_vacs_occs].values[j]+x_alt, vacs[manuf_vacs_occs][j]+y_alt),
#                     arrowprops=dict(arrowstyle= '->', color='black', lw=1),
                    va='bottom', ha=ha, fontsize=10, c='black', )#fontweight='bold')  
    elif (j==5):
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data.iloc[manuf_vacs_occs].values[j], vacs[manuf_vacs_occs][j]+4*y_alt), 
#                     xytext=(x_data.iloc[manuf_vacs_occs].values[j]+x_alt, vacs[manuf_vacs_occs][j]+y_alt),
#                     arrowprops=dict(arrowstyle= '->', color='black', lw=1),
                    va='bottom', ha=ha, fontsize=10, c='black', )#fontweight='bold')  
    elif (j==2):
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data.iloc[manuf_vacs_occs].values[j], vacs[manuf_vacs_occs][j]-y_alt), 
                    xytext=(x_data.iloc[manuf_vacs_occs].values[j]+4*x_alt, vacs[manuf_vacs_occs][j]-0.35),
                    arrowprops=dict(arrowstyle= '->', color='black', lw=1),
                    va='bottom', ha=ha, fontsize=10, c='black', )#fontweight='bold')  
    elif (j==3):
        axs.annotate(' ',
                        xy=(x_data.iloc[manuf_vacs_occs].values[j]+x_alt/2, vacs[manuf_vacs_occs][j]-y_alt), 
                    xytext=(x_data.iloc[manuf_vacs_occs].values[j]+5*x_alt, vacs[manuf_vacs_occs][j]-0.18),
                    arrowprops=dict(arrowstyle= '->', color='black', lw=1),
                    va='bottom', ha=ha, fontsize=10, c='black', )#fontweight='bold')  
    elif (j==6):
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data.iloc[manuf_vacs_occs].values[j]-x_alt, vacs[manuf_vacs_occs][j]), 
#                     xytext=(x_data.iloc[manuf_vacs_occs].values[j]+x_alt, vacs[manuf_vacs_occs][j]+y_alt),
#                     arrowprops=dict(arrowstyle= '->', color='black', lw=1),
                    va=va, ha='right', fontsize=10, c='black', )#fontweight='bold')  
    elif (j==7):
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data.iloc[manuf_vacs_occs].values[j], vacs[manuf_vacs_occs][j]+3*y_alt), 
#                     xytext=(x_data.iloc[manuf_vacs_occs].values[j]+x_alt, vacs[manuf_vacs_occs][j]+y_alt),
#                     arrowprops=dict(arrowstyle= '->', color='black', lw=1),
                    va='bottom', ha='right', fontsize=10, c='black', )#fontweight='bold')  
    elif (j==8):
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data.iloc[manuf_vacs_occs].values[j]+x_alt, vacs[manuf_vacs_occs][j]), 
#                     xytext=(x_data.iloc[manuf_vacs_occs].values[j]+x_alt, vacs[manuf_vacs_occs][j]+y_alt),
#                     arrowprops=dict(arrowstyle= '->', color='black', lw=1),
                    va='top', ha='left', fontsize=10, c='black', )#fontweight='bold')  
    elif (j==9):
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data.iloc[manuf_vacs_occs].values[j]+x_alt/2, vacs[manuf_vacs_occs][j]-y_alt), 
                    xytext=(x_data.iloc[manuf_vacs_occs].values[j]+4, vacs[manuf_vacs_occs][j]-0.15),
                    arrowprops=dict(arrowstyle= '->', color='black', lw=1, relpos=(0.2,1)),
                    va='bottom', ha='left', fontsize=10, c='black', )#fontweight='bold')  
    elif (j==10):
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data.iloc[manuf_vacs_occs].values[j]+x_alt/2, vacs[manuf_vacs_occs][j]-y_alt), 
                    xytext=(x_data.iloc[manuf_vacs_occs].values[j]+4, vacs[manuf_vacs_occs][j]-0.115),
                    arrowprops=dict(arrowstyle= '->', color='black', lw=1, relpos=(0.1,0.8)),
                    va='bottom', ha='left', fontsize=10, c='black', )#fontweight='bold')  
    elif (j==11):
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data.iloc[manuf_vacs_occs].values[j], vacs[manuf_vacs_occs][j]+y_alt), 
                    xytext=(x_data.iloc[manuf_vacs_occs].values[j]-1, vacs[manuf_vacs_occs][j]+0.22),
                    arrowprops=dict(arrowstyle= '->', color='black', lw=1, ),#relpos=(0.1,0.8)),
                    va='bottom', ha='right', fontsize=10, c='black', )#fontweight='bold')  
    elif (j==4):
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data.iloc[manuf_vacs_occs].values[j]-0.5, vacs[manuf_vacs_occs][j]), 
                    xytext=(x_data.iloc[manuf_vacs_occs].values[j]-2, vacs[manuf_vacs_occs][j]+0.01),
                    arrowprops=dict(arrowstyle= '->', color='black', lw=1, relpos=(1,0.5)),
                    va='bottom', ha='right', fontsize=10, c='black', )#fontweight='bold')  
    elif (j==1):
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data.iloc[manuf_vacs_occs].values[j], vacs[manuf_vacs_occs][j]), 
                    xytext=(x_data.iloc[manuf_vacs_occs].values[j]-1.5, vacs[manuf_vacs_occs][j]+0.07),
                    arrowprops=dict(arrowstyle= '->', color='black', lw=1, relpos=(1,0.5)),
                    va='bottom', ha='right', fontsize=10, c='black', )#fontweight='bold')  
    else:
        axs.annotate(short_dict[results_df['full_name'].iloc[occ_code]],
                        xy=(x_data.iloc[manuf_vacs_occs].values[j]+x_alt, vacs[manuf_vacs_occs][j]+y_alt), 
#                     xytext=(x_data.iloc[manuf_vacs_occs].values[j]+x_alt, vacs[manuf_vacs_occs][j]+y_alt),
#                     arrowprops=dict(arrowstyle= '->', color='black', lw=1),
                    va=va, ha='right', fontsize=10, c='black', )#fontweight='bold')  

  
if save == True:
    plt.savefig('figures/vacancy_manuf.pdf',bbox_inches='tight')
    plt.savefig('figures/vacancy_manuf.png',bbox_inches='tight')    


# plt.show()

In [ ]:
large_occs = list(np.where(employment_0 >= 1000)[0])

In [ ]:
len(large_occs)

In [ ]:
results_df['vacs_agr'] = vacs_0
results_df['vacs_manuf'] = vacs

In [ ]:
results_df.iloc[large_occs].nlargest(5, 'vacs_agr')['full_name'].values


In [ ]:
results_df.iloc[large_occs].nlargest(5, 'vacs_manuf')['full_name'].values


In [ ]:
## 1-digit vacancy rate
# results_dict_norm_to_bl['Cen2BAseq']['U']
results_dict_1digit = {}
for scn in scenarios:
    results_dict_1digit[scn] = {}
    for var in ['E', 'U', 'V', 'V_lt', 'D']:
        results_dict_1digit[scn][var] = np.array([np.sum(results_dict_norm_to_bl[scn][var]
                                                         [:,np.where((results_df['occ_1'] == digit)&
                                                                     (results_df['emp_0']>1000))[0]], axis=1) 
                                                  for digit in range(0,10)]).T

In [ ]:
cbo_1 = pd.read_csv(data_path+'CBO2002_occ_titles_1digit.csv')
cbo_1.loc[0,'TITLE'] = 'Managers'

In [ ]:
cbo_1.loc[-1] = [0, 'Military workers']  # adding a row
cbo_1.index = cbo_1.index + 1  # shifting index
cbo_1 = cbo_1.sort_index()  # sorting by index

In [ ]:
cbo_1

In [ ]:
xlabels_1digit = [re.sub("(.{"+str(i)+"})", "\\1\n", label, 1, re.DOTALL) for i, label in 
           zip([30, 30,25,30,30,27,27,21,21,22], list(cbo_1['TITLE']))]

In [ ]:
## 4-digit 
# results_dict_norm_to_bl['Cen2BAseq']['U']
results_dict_4digit = {}

for scn in scenarios:
    results_dict_4digit[scn] = {}
    
    for var in ['E', 'U', 'V', 'V_lt', 'D']:
        results_dict_4digit[scn][var] = np.array([np.sum(results_dict_norm_to_bl[scn][var]
                                                         [:,np.where((results_df['occ_code'] == digit)&
                                                                     (results_df['emp_0']>1000))[0]], axis=1) 
                                                  for digit in np.sort(list(results_df['occ_code'].unique())) ]).T


In [ ]:
# results_dict_norm_to_bl['Cen2BAseq']['U']
results_dict_reg = {}
for scn in scenarios:
    results_dict_reg[scn] = {}
    for var in ['E', 'U', 'V', 'V_lt', 'D']:
        results_dict_reg[scn][var] = np.array([np.sum(results_dict_norm_to_bl[scn][var]
                                                         [:,np.where((results_df['loc_code'] == digit)&
                                                                     (results_df['emp_0']>1000))[0]], axis=1) 
                                                  for digit in range(16)]).T

In [ ]:
t_start = 260
t_end = 416

In [ ]:
## create new plot:
# baseline vacancy rate ------ scenario vacancy rate
vacs_rate_bl = period_vac_rate(results_dict_norm_to_bl['Cen2BAseq']['V']+
                               results_dict_norm_to_bl['Cen2BAseq']['E'], 
                               results_dict_norm_to_bl['Cen2BAseq']['V_lt'], t_start, t_end)

vacs_rate_0 = period_vac_rate(results_dict_norm_to_bl['Cen2Aseq']['V']+
                               results_dict_norm_to_bl['Cen2Aseq']['E'], 
                               results_dict_norm_to_bl['Cen2Aseq']['V_lt'], t_start, t_end)

vacs_rate = period_vac_rate(results_dict_norm_to_bl['Cen5Aseq']['V']+
                               results_dict_norm_to_bl['Cen5Aseq']['E'], 
                               results_dict_norm_to_bl['Cen5Aseq']['V_lt'], t_start, t_end)

In [ ]:
## create new plot:
vacs_rate_bl_1digit = period_vac_rate(results_dict_1digit['Cen2BAseq']['V']+
                               results_dict_1digit['Cen2BAseq']['E'], 
                               results_dict_1digit['Cen2BAseq']['V_lt'], t_start, t_end)

vacs_rate_0_1digit = period_vac_rate(results_dict_1digit['Cen2Aseq']['V']+
                               results_dict_1digit['Cen2Aseq']['E'], 
                               results_dict_1digit['Cen2Aseq']['V_lt'], t_start, t_end)

vacs_rate_1digit = period_vac_rate(results_dict_1digit['Cen5Aseq']['V']+
                               results_dict_1digit['Cen5Aseq']['E'], 
                               results_dict_1digit['Cen5Aseq']['V_lt'], t_start, t_end)

In [ ]:
emp_per_1digit_occ = np.array([np.sum(results_df_large_occs_only[
                    results_df_large_occs_only['occ_1'] == i]['emp_0']) for i in range(0,10)])

In [ ]:
vacs_rate_0_1digit_perc = np.array(100*(vacs_rate_0_1digit-vacs_rate_bl_1digit) / vacs_rate_bl_1digit)
vacs_rate_1digit_perc = np.array(100*(vacs_rate_1digit-vacs_rate_bl_1digit) / vacs_rate_bl_1digit)

In [ ]:
## create new plot:
# baseline vacancy rate ------ scenario vacancy rate
vacs_rate_bl_reg = period_vac_rate(results_dict_reg['Cen2BAseq']['V']+
                               results_dict_reg['Cen2BAseq']['E'], 
                               results_dict_reg['Cen2BAseq']['V_lt'], t_start, t_end)

vacs_rate_0_reg = period_vac_rate(results_dict_reg['Cen2Aseq']['V']+
                               results_dict_reg['Cen2Aseq']['E'], 
                               results_dict_reg['Cen2Aseq']['V_lt'], t_start, t_end)

vacs_rate_reg = period_vac_rate(results_dict_reg['Cen5Aseq']['V']+
                               results_dict_reg['Cen5Aseq']['E'], 
                               results_dict_reg['Cen5Aseq']['V_lt'], t_start, t_end)

In [ ]:
emp_per_region = np.array([np.sum(results_df_large_occs_only[results_df_large_occs_only['loc_code'] == i]['emp_0']) 
                           for i in range(16)])

In [ ]:
no_occs_per_region = np.array([len(results_df_large_occs_only[results_df_large_occs_only['loc_code'] == i])
                               for i in range(16)])

In [ ]:
vacs_rate_0_reg_perc = np.array(100*(vacs_rate_0_reg-vacs_rate_bl_reg) / vacs_rate_bl_reg)
vacs_rate_reg_perc = np.array(100*(vacs_rate_reg-vacs_rate_bl_reg) / vacs_rate_bl_reg)

In [ ]:
# results_dict_norm_to_bl['Cen2BAseq']['U']
results_dict_reg1digit = {}
for scn in scenarios:
    results_dict_reg1digit[scn] = {}
    for reg in regions:
        results_dict_reg1digit[scn][reg] = {}
        for var in ['E', 'U', 'V', 'V_lt', 'D']:
            results_dict_reg1digit[scn][reg][var] = np.array([np.sum(results_dict_norm_to_bl[scn][var]
                                                             [:,np.where((results_df['occ_1'] == digit)&(
                                                             results_df['loc'] == reg))[0]], axis=1) 
                                                      for digit in range(0,10)]).T


In [ ]:
unemp_rate_bl_reg1digit = np.array([period_occ_u_rate(
                               results_dict_reg1digit['Cen2BAseq'][reg]['E'], 
                               results_dict_reg1digit['Cen2BAseq'][reg]['U'], t_start, t_end) 
                          for reg in regions])

unemp_rate_0_reg1digit = np.array([period_occ_u_rate(
                               results_dict_reg1digit['Cen2Aseq'][reg]['E'], 
                               results_dict_reg1digit['Cen2Aseq'][reg]['U'], t_start, t_end)
                         for reg in regions])

unemp_rate_reg1digit = np.array([period_occ_u_rate(
                               results_dict_reg1digit['Cen5Aseq'][reg]['E'], 
                               results_dict_reg1digit['Cen5Aseq'][reg]['U'], t_start, t_end)
                       for reg in regions])

In [ ]:
vmin = np.min((unemp_rate_0_reg1digit-unemp_rate_bl_reg1digit,unemp_rate_reg1digit-unemp_rate_bl_reg1digit))
vmax = np.max((unemp_rate_0_reg1digit-unemp_rate_bl_reg1digit, unemp_rate_reg1digit-unemp_rate_bl_reg1digit))

# vmm = [vmin, vmax][np.argmax([abs(1-vmin), abs(1-vmax)])]
print(vmin, vmax)
# if vmm >1:
#     vmin=2-vmm
#     vmax=vmm
# else:
#     vmin=vmm
#     vmax=2-vmm

In [ ]:
# USD gdp from Wikipedia - IGBE 2016 Regional Accounts Report
# joined regions added together
gdp_regions = [3.934,  4.102,               25.468,                                    78.127, 
       51.982+67.376,  (9.033+24.400+11.846), 26.283,                                    35.429,
       39.501,         114.916,             (17.069+39.590+16.905+47.861+14.149+11.120), 
               (155.821+31.250+183.158),
       116.914+73.431, 11.287,              3.150,               583.077, 
              ]

In [ ]:
# USD gdp from Wikipedia - IGBE 2016 Regional Accounts Report
# joined regions added together
gdp_pc_regions = [3.934/0.8,  4.102/0.8,               25.468/4,      78.127/15.3, 
       (51.982+67.376)/(6.7+3),  (9.033+24.400+11.846)/(1.5+7+3.2), 26.283/2.7,
                   35.429/3.3,
       39.501/8.3,         114.916/11.2,   
         (17.069+39.590+16.905+47.861+14.149+11.120)/(3.5+9+4+9.4+3.4+2.3), 
               (155.821+31.250+183.158)/(20.9+3.9+16.6),
       (116.914+73.431)/(11.3+6.9),    11.287/1.8,     3.150/0.5,     583.077/44.8, 
              ]

In [ ]:
mean_wage_per_1digit = pd.DataFrame([9227.783711, 5401.866298, 4828.684458, 2686.807683, 1841.363321, 
                        1774.235092, 1557.419791, 1914.425304, 2237.693771, 2343.606856], columns=['occ_1']).occ_1


In [ ]:
row_idxs = np.argsort(mean_wage_per_1digit)[::-1]# * -1)
col_idxs = np.argsort(gdp_pc_regions)[::-1]# * -1)


In [ ]:
cbo_1['TITLE_short'] = ['Military workers', 'Managers',
 'Professionals of sciences and arts',
 'Middle-level technicians',
 'Administrative workers',
 'Service workers',
 'Agricultural workers',
 'Industrial goods  workers (craft)',
 'Industrial goods workers (machine)',
 'Repair and maintenance workers']

In [ ]:
## Split into two
f, axs = plt.subplots(1,1, figsize=(7,5))
f.tight_layout(pad = 4.5)

title_fontsize = 14
axislabels_fontsize = 14
small_fontsize = 10

vmin=-0.4
vmax=0.4

row_idxs = np.argsort(mean_wage_per_1digit)[::-1]# * -1)
col_idxs = np.argsort(gdp_pc_regions)[::-1]# * -1)


uplot = axs.imshow((unemp_rate_0_reg1digit-unemp_rate_bl_reg1digit).T[row_idxs.drop(9),:][:, col_idxs], cmap='RdBu_r',
             vmin=vmin, vmax=vmax, 
             )
# axs.set_title('Agriculture Scenario', fontsize=title_fontsize)

axs.set_xticks(range(16))
axs.set_xticklabels(np.array(regions)[col_idxs], rotation=45, ha='right', fontsize=axislabels_fontsize, rotation_mode='anchor')

axs.set_yticks(range(9))
axs.set_yticklabels(cbo_1['TITLE_short'][row_idxs.drop(9)], fontsize=axislabels_fontsize)

cbar = plt.colorbar(uplot, ax=axs, fraction=0.03, pad=0.12)
cbar.ax.tick_params(labelsize=axislabels_fontsize)
cbar.set_label(label='2018-2030 Average unemployment\nrate change from baseline (%-point)',
              fontsize=axislabels_fontsize)


plt.gcf().text(0.82, 0.8, 'Mean\nwage\n(BRL)', fontsize=14, ha='center')
for i in range(0,9):
    plt.gcf().text(0.79, 0.275+i/17.5, int(np.round(np.sort(mean_wage_per_1digit), -1)[i]), fontsize=14)


plt.gcf().text(0.13, 0.02, 'High GDP', fontsize=axislabels_fontsize)
plt.gcf().text(0.7, 0.02, 'Low GDP', fontsize=axislabels_fontsize)
# plt.arrow(10,-1,-10,1)
axs.annotate('', xy=(0.22, -0.475), xycoords='axes fraction', xytext=(0.84, -0.475), 
            arrowprops=dict(arrowstyle="<-", color='black'))

if save == True:
    plt.savefig('figures/unemp_heatmap_agr.pdf', bbox_inches='tight')
    plt.savefig('figures/unemp_heatmap_agr.png', bbox_inches='tight')
plt.show()




In [ ]:
## Split into two
f, axs = plt.subplots(1,1, figsize=(7,5))
f.tight_layout(pad = 4.5)

title_fontsize = 14
axislabels_fontsize = 14
small_fontsize = 10

vmin=-0.4
vmax=0.4

row_idxs = np.argsort(mean_wage_per_1digit)[::-1]# * -1)
col_idxs = np.argsort(gdp_pc_regions)[::-1]# * -1)


uplot = axs.imshow((unemp_rate_reg1digit-unemp_rate_bl_reg1digit).T[row_idxs.drop(9),:][:, col_idxs], cmap='RdBu_r',
             vmin=vmin, vmax=vmax, 
             )
# axs.set_title('Agriculture Scenario', fontsize=title_fontsize)

axs.set_xticks(range(16))
axs.set_xticklabels(np.array(regions)[col_idxs], rotation=45, ha='right', fontsize=axislabels_fontsize, rotation_mode='anchor')

axs.set_yticks(range(9))
axs.set_yticklabels(cbo_1['TITLE_short'][row_idxs.drop(9)], fontsize=axislabels_fontsize)

cbar = plt.colorbar(uplot, ax=axs, fraction=0.03, pad=0.12)
cbar.ax.tick_params(labelsize=axislabels_fontsize)
cbar.set_label(label='2018-2030 Average unemployment\nrate change from baseline (%-point)',
              fontsize=axislabels_fontsize)

# plt.gcf().text(0.8, 0.68, 'high\nwage', fontsize=axislabels_fontsize)
# plt.gcf().text(0.8, 0.27, 'low\nwage', fontsize=axislabels_fontsize)
# # plt.arrow(10,-1,-10,1)
# axs.annotate('', xy=(1.05, 0.8), xycoords='axes fraction', xytext=(1.05, 0.15), 
#             arrowprops=dict(arrowstyle="<-", color='black'))


plt.gcf().text(0.83, 0.74, 'High\nwage', fontsize=13, ha='center')
plt.gcf().text(0.83, 0.25, 'Low\nwage', fontsize=13, ha='center')
# for i in range(10):
#     plt.gcf().text(0.8, 0.25+i/17.5, int(np.round(np.sort(mean_wage_per_1digit), -1)[i]), fontsize=12)
axs.annotate('', xy=(1.06, 0.87), xycoords='axes fraction', xytext=(1.06, 0.15), 
            arrowprops=dict(arrowstyle="<-", color='black'))

plt.gcf().text(0.13, 0.02, 'High GDP', fontsize=axislabels_fontsize)
plt.gcf().text(0.7, 0.02, 'Low GDP', fontsize=axislabels_fontsize)
# plt.arrow(10,-1,-10,1)
axs.annotate('', xy=(0.22, -0.475), xycoords='axes fraction', xytext=(0.84, -0.475), 
            arrowprops=dict(arrowstyle="<-", color='black'))

if save == True:
    plt.savefig('figures/unemp_heatmap_manuf.pdf', bbox_inches='tight')
    plt.savefig('figures/unemp_heatmap_manuf.png', bbox_inches='tight')
plt.show()




In [ ]:
vacs_rate_bl_reg1digit = np.array([period_vac_rate(results_dict_reg1digit['Cen2BAseq'][reg]['V']+
                               results_dict_reg1digit['Cen2BAseq'][reg]['E'], 
                               results_dict_reg1digit['Cen2BAseq'][reg]['V_lt'], t_start, t_end) 
                          for reg in regions])

vacs_rate_0_reg1digit = np.array([period_vac_rate(results_dict_reg1digit['Cen2Aseq'][reg]['V']+
                               results_dict_reg1digit['Cen2Aseq'][reg]['E'], 
                               results_dict_reg1digit['Cen2Aseq'][reg]['V_lt'], t_start, t_end)
                         for reg in regions])

vacs_rate_reg1digit = np.array([period_vac_rate(results_dict_reg1digit['Cen5Aseq'][reg]['V']+
                               results_dict_reg1digit['Cen5Aseq'][reg]['E'], 
                               results_dict_reg1digit['Cen5Aseq'][reg]['V_lt'], t_start, t_end)
                       for reg in regions])



In [ ]:
save = True

In [ ]:
## Split into two
f, axs = plt.subplots(1,1, figsize=(7,5))
f.tight_layout(pad = 4.5)

title_fontsize = 14
axislabels_fontsize = 14
small_fontsize = 10

vmin=-0.65
vmax=0.65

row_idxs = np.argsort(mean_wage_per_1digit)[::-1]# * -1)
col_idxs = np.argsort(gdp_pc_regions)[::-1]# * -1)


uplot = axs.imshow((vacs_rate_0_reg1digit-vacs_rate_bl_reg1digit).T[row_idxs.drop(9),:][:, col_idxs], cmap='PuOr',
             vmin=vmin, vmax=vmax, 
             )
# axs.set_title('Agriculture Scenario', fontsize=title_fontsize)

axs.set_xticks(range(16))
axs.set_xticklabels(np.array(regions)[col_idxs], rotation=45, ha='right', fontsize=axislabels_fontsize, rotation_mode='anchor')

axs.set_yticks(range(9))
axs.set_yticklabels(cbo_1['TITLE_short'][row_idxs.drop(9)], fontsize=axislabels_fontsize)

cbar = plt.colorbar(uplot, ax=axs, fraction=0.03, pad=0.12)
cbar.ax.tick_params(labelsize=axislabels_fontsize)
cbar.set_label(label='Average 6-month unfilled\nvacancy rate change from the\nbaseline 2018-2030 (%-point)',
              fontsize=axislabels_fontsize)


plt.gcf().text(0.82, 0.8, 'Mean\nwage\n(BRL)', fontsize=14, ha='center')
for i in range(0,9):
    plt.gcf().text(0.79, 0.275+i/17.5, int(np.round(np.sort(mean_wage_per_1digit), -1)[i]), fontsize=14)


plt.gcf().text(0.13, 0.02, 'High GDP', fontsize=axislabels_fontsize)
plt.gcf().text(0.7, 0.02, 'Low GDP', fontsize=axislabels_fontsize)
# plt.arrow(10,-1,-10,1)
axs.annotate('', xy=(0.22, -0.475), xycoords='axes fraction', xytext=(0.84, -0.475), 
            arrowprops=dict(arrowstyle="<-", color='black'))

if save == True:
    plt.savefig('figures/vacancy_heatmap_agr.pdf', bbox_inches='tight')
    plt.savefig('figures/vacancy_heatmap_agr.png', bbox_inches='tight')
plt.show()




In [ ]:
## Split into two
f, axs = plt.subplots(1,1, figsize=(7,5))
f.tight_layout(pad = 4.5)

title_fontsize = 14
axislabels_fontsize = 14
small_fontsize = 10

vmin=-0.65
vmax=0.65

row_idxs = np.argsort(mean_wage_per_1digit)[::-1]# * -1)
col_idxs = np.argsort(gdp_pc_regions)[::-1]# * -1)


uplot = axs.imshow((vacs_rate_reg1digit-vacs_rate_bl_reg1digit).T[row_idxs.drop(9),:][:, col_idxs], cmap='PuOr',
             vmin=vmin, vmax=vmax, 
             )
# axs.set_title('Agriculture Scenario', fontsize=title_fontsize)

axs.set_xticks(range(16))
axs.set_xticklabels(np.array(regions)[col_idxs], rotation=45, ha='right', fontsize=axislabels_fontsize, rotation_mode='anchor')

axs.set_yticks(range(9))
axs.set_yticklabels(cbo_1['TITLE_short'][row_idxs.drop(9)], fontsize=axislabels_fontsize)

cbar = plt.colorbar(uplot, ax=axs, fraction=0.03, pad=0.12)
cbar.ax.tick_params(labelsize=axislabels_fontsize)
cbar.set_label(label='Average 6-month unfilled\nvacancy rate change from the\nbaseline 2018-2030 (%-point)',
              fontsize=axislabels_fontsize)

# plt.gcf().text(0.8, 0.68, 'high\nwage', fontsize=axislabels_fontsize)
# plt.gcf().text(0.8, 0.27, 'low\nwage', fontsize=axislabels_fontsize)
# # plt.arrow(10,-1,-10,1)
# axs.annotate('', xy=(1.05, 0.8), xycoords='axes fraction', xytext=(1.05, 0.15), 
#             arrowprops=dict(arrowstyle="<-", color='black'))


plt.gcf().text(0.83, 0.74, 'High\nwage', fontsize=13, ha='center')
plt.gcf().text(0.83, 0.25, 'Low\nwage', fontsize=13, ha='center')
# for i in range(10):
#     plt.gcf().text(0.8, 0.25+i/17.5, int(np.round(np.sort(mean_wage_per_1digit), -1)[i]), fontsize=12)
axs.annotate('', xy=(1.06, 0.87), xycoords='axes fraction', xytext=(1.06, 0.15), 
            arrowprops=dict(arrowstyle="<-", color='black'))

plt.gcf().text(0.13, 0.02, 'High GDP', fontsize=axislabels_fontsize)
plt.gcf().text(0.7, 0.02, 'Low GDP', fontsize=axislabels_fontsize)
# plt.arrow(10,-1,-10,1)
axs.annotate('', xy=(0.22, -0.475), xycoords='axes fraction', xytext=(0.84, -0.475), 
            arrowprops=dict(arrowstyle="<-", color='black'))

if save == True:
    plt.savefig('figures/vacancy_heatmap_sorted_manuf.pdf', bbox_inches='tight')
    plt.savefig('figures/vacancy_heatmap_sorted_manuf.png', bbox_inches='tight')
plt.show()




In [ ]:
f, ax = plt.subplots(1,2, figsize=(7,3))
ax[0].bar(range(2), [100, 100], color='tab:olive', 
          label='between-\nregion')
ax[1].bar(range(2), [100, 100], color='tab:olive', 
          label='between-\nregion')

ax[0].bar(range(2), [(between_occ_un_manuf+residual_un_manuf)/total_var_un_manuf*100, 
                      (between_occ_vac_manuf+residual_vac_manuf)/total_var_vac_manuf*100], 
          color='tab:blue', label='between-\noccupation')
ax[1].bar(range(2), [(between_occ_un_agr+residual_un_agr)/total_var_un_agr*100,
                     (between_occ_vac_agr+residual_vac_agr)/total_var_vac_agr*100], 
          color='tab:blue', label='between-\noccupation')

ax[0].bar(range(2), [residual_un_manuf/total_var_un_manuf*100, residual_vac_manuf/total_var_vac_manuf*100], 
          color='gray', label='residual')
ax[1].bar(range(2), [residual_un_agr/total_var_un_agr*100, residual_vac_agr/total_var_vac_agr*100],
          color='gray', label='residual')

plt.legend(loc='upper left', bbox_to_anchor=(1.02, 1.05), title='Variance', fontsize=10, title_fontsize=10)

for i in range(2):
    ax[i].set_ylabel('%', fontsize=10 )
    ax[i].yaxis.set_label_coords(-0.12,0.5)
#     ax[i].set_xticks([0,1], ['2018-2030 Average \nunemployment rate\n change from baseline', 
#                              '2018-2030 Average\n unfilled vacancy rate\n change from baseline'], fontsize=14)
    ax[i].set_xticks([0,1], 
        ['Unemployment\nrate\n                              (average 2018-2030 change\n                          from baseline)', 
                             'Unfilled\nvacancy rate'], fontsize=10, linespacing=1.5)
#     ax[i].set_yticks([70,75,80,85,90,95,100], fontsize=20)
#     ax[i].set_ylim(70,101)

for i,vars_list in enumerate([[between_reg_un_manuf, between_occ_un_manuf,residual_un_manuf,total_var_un_manuf,
    between_reg_vac_manuf, between_occ_vac_manuf,residual_vac_manuf,total_var_vac_manuf],
                              [between_reg_un_agr, between_occ_un_agr,residual_un_agr,total_var_un_agr,
    between_reg_vac_agr, between_occ_vac_agr,residual_vac_agr,total_var_vac_agr]
    ]):
    k = 0
    for j in [0,4]:
    
        ax[i].annotate(str(np.round(vars_list[j]/vars_list[j+3]*100,1))+'%', 
               xy=(k,100-vars_list[j]/vars_list[j+3]*100/2),fontsize=10, 
               c='black', zorder=6, ha='center', va='center')#fontweight='bold')  
        
        ax[i].annotate(str(np.round(vars_list[j+1]/vars_list[j+3]*100,1))+'%', 
               xy=(k,vars_list[j+1]/vars_list[j+3]*100/2+vars_list[j+2]/vars_list[j+3]*100),fontsize=10, 
               c='white', zorder=6, ha='center')#fontweight='bold')  
        
        ax[i].annotate(str(np.round(vars_list[j+2]/vars_list[j+3]*100,1))+'%', 
               xy=(k,vars_list[j+2]/vars_list[j+3]*100/2),fontsize=10, 
               c='white', zorder=6, ha='center')#fontweight='bold')  
        k += 1

# ax[0].spines['top'].set_visible(False)
# ax[0].spines['right'].set_visible(False)

# ax[1].spines['top'].set_visible(False)
# ax[1].spines['right'].set_visible(False)
        
ax[0].set_title('Manufacturing growth path', fontsize=10)
ax[1].set_title('Agriculture growth path', fontsize=10)
if save == True:
    plt.savefig('figures/variance_decomposition.pdf', bbox_inches='tight')
    plt.savefig('figures/variance_decomposition.png', bbox_inches='tight')
plt.show()

## Reallocation through time

In [ ]:
# to get the unemployment rate we sum over all occupations and divide by labor
# force (x100 to make it percentage)
unemployment_rate = 100*results_dict_norm_to_bl['Cen2BAseq']['U'].sum(axis=1)/L
unemployment_rate_agr = 100*results_dict_norm_to_bl['Cen2Aseq']['U'].sum(axis=1)/L
unemployment_rate_manuf = 100*results_dict_norm_to_bl['Cen5Aseq']['U'].sum(axis=1)/L

fig, ax1 = plt.subplots(figsize=(4,3))

ax1.plot(unemployment_rate, label='Baseline', color='tab:blue')
ax1.plot(unemployment_rate_manuf, label='Manufacturing growth path', color='tab:orange')
ax1.plot(unemployment_rate_agr, label='Agriculture growth path', color='tab:green')
ax1.tick_params(axis='y', labelcolor='k')

# ax2 = ax1.twinx()

# cols = ['tab:blue', 'tab:green', 'tab:orange']
# markers = ['s', 'o', '^']


# handles, labels = ax.get_legend_handles_labels()
ax1.legend(loc='upper left')

# plt.title('Regional occupational mobility network')
plt.xlabel("Time")
plt.xlim(t_start,t_end)
plt.xticks(np.arange(t_start, t_end+1, 13), years, rotation=45)

ax1.set_ylim(6.2,6.4)
ax1.set_ylabel("Unemployment rate (%)")

# ax2.set_ylabel('Labour reallocation')
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)


if save == True:
    plt.savefig('figures/aggregate_unemp_omng.png', bbox_inches='tight')
    plt.savefig('figures/aggregate_unemp_omng.pdf', bbox_inches='tight')
plt.show()

In [ ]:
save = True

In [ ]:
scenarios

In [ ]:
plot_data_demand = pd.DataFrame()

In [ ]:
# to get the unemployment rate we sum over all occupations and divide by labor
# force (x100 to make it percentage)
unemployment_rate = 100*results_dict_norm_to_bl['Cen2BAseq']['U'].sum(axis=1)/L
unemployment_rate_agr = 100*results_dict_norm_to_bl['Cen2Aseq']['U'].sum(axis=1)/L
unemployment_rate_manuf = 100*results_dict_norm_to_bl['Cen5Aseq']['U'].sum(axis=1)/L

fig, ax1 = plt.subplots(figsize=(4,3))

# ax1.plot(unemployment_rate, label='Baseline', color='tab:blue')
# ax1.plot(unemployment_rate_manuf, label='Manufacturing growth path', color='tab:orange')
# ax1.plot(unemployment_rate_agr, label='Agriculture growth path', color='tab:green')
# ax1.tick_params(axis='y', labelcolor='k')

# ax2 = ax1.twinx()

cols = ['tab:blue', 'tab:orange', 'tab:green']
markers = ['s', 'o', '^']

for j, (scn, lab) in enumerate(zip(['Cen2BAseq', 'Cen5Aseq', 'Cen2Aseq'], ['Baseline', 'Manufacturing growth path',
                                                'Agriculture growth path'])):
## get entries where x_data_0 > 0 and per 1-digit ocupation 
## sum for each occupation
    increase_demand = np.zeros(12)
    for i, year in enumerate(years[:-1]):
        year_change = (scenario_dict_norm_to_bl[scn].loc[year+1]-scenario_dict_norm_to_bl[scn].loc[year]
              ).reset_index(drop=True)
        increase_demand[i] = np.sum(year_change.iloc[list(np.where((year_change>0)
                                                                     &(results_df['emp_0']>1000))[0])])

    ax1.scatter(np.arange(t_start+13/2, t_end, 13), increase_demand/(1e6), color=cols[j], marker=markers[j],
               label=lab)
    plot_data_demand[scn] = increase_demand

ax1.tick_params(axis='y', labelcolor='k')

# handles, labels = ax.get_legend_handles_labels()
ax1.legend()

# plt.title('Regional occupational mobility network')
plt.xlabel("Time")
plt.xlim(t_start,t_end)
plt.xticks(np.arange(t_start, t_end+1, 13), years, rotation=45)

# ax1.set_ylim(6.2,6.4)
# ax1.set_ylabel("Unemployment rate (%)")

ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)

ax1.set_ylabel('Labour reallocation (million jobs)')


if save == True:
    plt.savefig('figures/aggregate_reallocation.png', bbox_inches='tight')
    plt.savefig('figures/aggregate_reallocation.pdf', bbox_inches='tight')
plt.show()

## Maps

In [ ]:
region_df_for_pt = pd.DataFrame({'GDP per\ncapita': np.array(gdp_pc_regions).round(2),
#                                  'Higher-emissions\nscenario': np.array(reg_diff_0).astype(int),
#                                  'Lower-emissions\nscenario': np.array(reg_diff).astype(int), 
                                 'Agriculture\ngrowth path': unemp_rate_0_reg_perc.round(2), 
                                 'Manufacturing\ngrowth path': unemp_rate_reg_perc.round(2),
                                 'reg vacs agr': vacs_rate_0_reg_perc.round(2), 
                                 'reg vacs manuf': vacs_rate_reg_perc.round(2)}
             , index=regions)

In [ ]:
cols = region_df_for_pt.columns

In [ ]:
region_df_for_pt.index.name = 'Region'

In [ ]:
region_df_for_pt

In [ ]:
brazil_gdf = gpd.read_file(data_path+'brazil_shapefile/bra_admbnda_adm1_ibge_2020.shp')

In [ ]:
brazil_gdf

In [ ]:
regs_for_map = ['Acre', 'RNordeste', 'Amapa', 'Amazonas', 'Bahia', 'RNordeste', 'GoiasDF', 'RSudeste',
'GoiasDF', 'Matopiba', 'MtGrosso', 'MtGrSul', 'RSudeste', 'Para', 'RNordeste', 'Parana',
'RNordeste', 'Matopiba', 'RSudeste', 'RNordeste', 'RSul', 'Rondonia', 'Roraima', 'RSul', 
'SaoPaulo', 'RNordeste', 'Matopiba']

In [ ]:
brazil_gdf['model_regions'] = regs_for_map

In [ ]:
brazil_gdf

In [ ]:
region_df_for_pt

In [ ]:
# region_df_for_pt = pd.DataFrame({'GDP per\ncapita': np.array(gdp_pc_regions).round(2),
# #                                  'Higher-emissions\nscenario': np.array(reg_diff_0).astype(int),
# #                                  'Lower-emissions\nscenario': np.array(reg_diff).astype(int), 
#                                  'Higher-\nemissions': unemp_rate_0_reg_perc.round(2), 
#                                  'Lower-\nemissions': unemp_rate_reg_perc.round(2),
#                                  'reg vacs agr': vacs_rate_0_reg_perc.round(2), 
#                                  'reg vacs manuf': vacs_rate_reg_perc.round(2)}
#              , index=regions)
brazil_gdf['y_agr'] = [region_df_for_pt.loc[reg, 'Agriculture\ngrowth path'] for reg in regs_for_map]
brazil_gdf['y_manuf'] = [region_df_for_pt.loc[reg, 'Manufacturing\ngrowth path'] for reg in regs_for_map]
brazil_gdf['vacs_agr'] = [region_df_for_pt.loc[reg, 'reg vacs agr'] for reg in regs_for_map]
brazil_gdf['vacs_manuf'] = [region_df_for_pt.loc[reg, 'reg vacs manuf'] for reg in regs_for_map]

In [ ]:
region_df_for_pt.head()

In [ ]:
# save = True

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(4,4))
ax = brazil_gdf.plot(column='y_agr', legend=True, categorical=False, cmap='RdBu_r', 
               legend_kwds={'label':'2018-2030 Average\nunemployment rate change\nfrom baseline (%-point)', 
                            'orientation': 'horizontal',                            'shrink':0.8, 
                            'pad':0.02
                           }, ax=ax, vmin=-np.max(unemp_rate_0_reg_perc), 
                     vmax=np.max(unemp_rate_0_reg_perc), 
                     edgecolor='black', linewidth=0.1)
ax.axis('off')
cb_ax = fig.axes[1]
cb_ax.tick_params(labelsize=10, labelcolor='black')
# plt.title('Agriculture growth path')
if save == True:
    plt.savefig('figures/unemp_map_agr.pdf', bbox_inches='tight')
    plt.savefig('figures/unemp_map_agr.png', bbox_inches='tight')
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(4,4))
ax = brazil_gdf.plot(column='y_manuf', legend=True, categorical=False, cmap='RdBu_r', 
               legend_kwds={'label':'2018-2030 Average\nunemployment rate change\nfrom baseline (%-point)',
                            'orientation': 'horizontal', 
                           'shrink':0.8, 
                            'pad':0.02
                           }, ax=ax, 
                     vmin=-np.max(unemp_rate_0_reg_perc), 
                     vmax=np.max(unemp_rate_0_reg_perc),
                     edgecolor='black', linewidth=0.1)
ax.axis('off')
cb_ax = fig.axes[1]
cb_ax.tick_params(labelsize=10, labelcolor='black')
# plt.title('Manufacturing growth path')
if save == True:
    plt.savefig('figures/unemp_map_manuf.pdf', bbox_inches='tight')
    plt.savefig('figures/unemp_map_manuf.png', bbox_inches='tight')
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(4,4))
ax = brazil_gdf.plot(column='vacs_agr', legend=True, categorical=False, cmap='PuOr', 
               legend_kwds={'label':'2018-2030 Average 6-month\nunfilled vacancy rate change\nfrom baseline (%-point)', 'orientation': 'horizontal', 
                           'shrink':0.8, 
                            'pad':0.02}, ax=ax, 
                     vmin=-np.max(vacs_rate_0_reg_perc), 
                     vmax=np.max(vacs_rate_0_reg_perc), 
                     edgecolor='black', linewidth=0.1)
ax.axis('off')
cb_ax = fig.axes[1]
cb_ax.tick_params(labelsize=10, labelcolor='black')
# plt.title('Agriculture growth path')
if save == True:
    plt.savefig('figures/vacancy_map_agr.pdf', bbox_inches='tight')
    plt.savefig('figures/vacancy_map_agr.png', bbox_inches='tight')
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(4,4))
ax = brazil_gdf.plot(column='vacs_manuf', legend=True, categorical=False, cmap='PuOr', 
               legend_kwds={'orientation': 'horizontal', 
                            'label':'2018-2030 Average 6-month\nunfilled vacancy rate change\nfrom baseline (%-point)',
                           'shrink':0.8, 
                            'pad':0.02
                           }, 
                     ax=ax, 
                     vmin=-np.max(vacs_rate_0_reg_perc), 
                     vmax=np.max(vacs_rate_0_reg_perc),
                     edgecolor='black', linewidth=0.1)
ax.axis('off')
cb_ax = fig.axes[1]
cb_ax.tick_params(labelsize=10, labelcolor='black')
# cb_ax.set_title(label='2018-2030 Average 6-month\nunfilled vacancy rate change\nfrom baseline (%-point)', 
#                 fontdict={'fontsize':8},, va='top')
# plt.title('Manufacturing growth path')
if save == True:
    plt.savefig('figures/vacancy_map_manuf.pdf', bbox_inches='tight')
    plt.savefig('figures/vacancy_map_manuf.png', bbox_inches='tight')
plt.show()## table

## Occupation table

In [ ]:
# ## create new plot:
unemp_rate_bl_1digit = period_occ_u_rate(results_dict_1digit['Cen2BAseq']['E'],
                               results_dict_1digit['Cen2BAseq']['U'], t_start, t_end)

unemp_rate_0_1digit = period_occ_u_rate(results_dict_1digit['Cen2Aseq']['E'],
                               results_dict_1digit['Cen2Aseq']['U'], t_start, t_end)

unemp_rate_1digit = period_occ_u_rate(results_dict_1digit['Cen5Aseq']['E'],
                               results_dict_1digit['Cen5Aseq']['U'], t_start, t_end)
unemp_rate_0_1digit_perc = np.array(100*(unemp_rate_0_1digit-unemp_rate_bl_1digit) / unemp_rate_bl_1digit)
unemp_rate_1digit_perc = np.array(100*(unemp_rate_1digit-unemp_rate_bl_1digit) / unemp_rate_bl_1digit)

# ## create new plot:
# # baseline vacancy rate ------ scenario vacancy rate
unemp_rate_bl_reg = period_occ_u_rate(results_dict_reg['Cen2BAseq']['E'],
                               results_dict_reg['Cen2BAseq']['U'], t_start, t_end)

unemp_rate_0_reg = period_occ_u_rate(results_dict_reg['Cen2Aseq']['E'],
                               results_dict_reg['Cen2Aseq']['U'], t_start, t_end)

unemp_rate_reg = period_occ_u_rate(results_dict_reg['Cen5Aseq']['E'],
                               results_dict_reg['Cen5Aseq']['U'], t_start, t_end)
unemp_rate_0_reg_perc = np.array(100*(unemp_rate_0_reg-unemp_rate_bl_reg) / unemp_rate_bl_reg)
unemp_rate_reg_perc = np.array(100*(unemp_rate_reg-unemp_rate_bl_reg) / unemp_rate_bl_reg)
# pd.DataFrame({'occ_1 unemp agr': unemp_rate_0_1digit_perc[row_idxs], 
#               'occ_1 unemp manuf': unemp_rate_1digit_perc[row_idxs], 
#               #'reg vacs agr': vacs_rate_0_reg_perc, 'reg vacs manuf': vacs_rate_reg_perc}
#              }, index=cbo_1.TITLE[row_idxs]).style.background_gradient(cmap='RdBu_r', 
#                                                                 vmin=-2.777843, vmax=2.777843)
                


In [ ]:
## table

from plottable import ColumnDefinition, Table
from plottable.cmap import normed_cmap, centered_cmap
from plottable.formatters import decimal_to_percent
from plottable.plots import circled_image, bar # image


In [ ]:
cbo_1['TITLE_wrapped'] = ['Military workers', 'Managers',
 'Professionals of\nsciences and arts',
 'Middle-level\ntechnicians',
 'Administrative\nworkers',
 'Service workers',
 'Agricultural, forestry and\nfisheries workers',
 'Industrial goods and\nservices workers (craft)',
 'Industrial goods and\nservices workers (machine)',
 'Repair and\nmaintenance workers']

In [ ]:
cbo_1['TITLE_short'] = ['Military workers', 'Managers',
 'Professionals of sciences and arts',
 'Middle-level technicians',
 'Administrative workers',
 'Service workers',
 'Agricultural workers',
 'Industrial goods  workers (craft)',
 'Industrial goods workers (machine)',
 'Repair and maintenance workers']

In [ ]:
len(vacs_rate_0_1digit_perc.round(2))

In [ ]:
occ1_df_for_pt = pd.DataFrame({'Mean wage\n(BRL)': np.array(mean_wage_per_1digit).astype(int),
#                                  'Higher-emissions\nscenario': np.array(occ_1_diff_0).astype(int),
#                                  'Lower-emissions\nscenario': np.array(occ_1_diff).astype(int), 
                                 
                                 'Manufacturing\ngrowth path': unemp_rate_1digit_perc.round(2),
                               'Agriculture\ngrowth path': unemp_rate_0_1digit_perc.round(2), 
                                 'reg\nmanuf': vacs_rate_1digit_perc.round(2), 
                                 'reg\nagr': vacs_rate_0_1digit_perc.round(2)}
             , index=cbo_1.TITLE_wrapped)
occ1_df_for_pt.index.name = 'Occupational\nGroup'

In [ ]:
occ1_df_for_pt.drop('Military workers', inplace=True)

In [ ]:
occ1_df_for_pt

In [ ]:
alph = 1

group_dict = {'unemp': '2018-2030 Average\nunemployment\nrate change from\nbaseline (%-point)',
            'vacs': '2018-2030 Average\n6-month unfilled vacancy\nrate change from\nbaseline (%-point)',
            'demand': 'Demand change\nfrom baseline in 2030'
            }

In [ ]:
col_defs = (
    [
        ColumnDefinition(
            name="Occupational\nGroup",
            textprops={"ha": "left", "weight": "bold"},
            width=2.8,
        ),
        ColumnDefinition(
            name="Mean wage\n(BRL)",
            textprops={"ha": "center"},
            formatter="{:,}",
#             width=0.75,
        ),
        ColumnDefinition(
            name="Agriculture\ngrowth path",
#             width=0.75,
            textprops={
                "ha": "center",
                "bbox": {"boxstyle": "round", "pad": 0.35, "alpha":alph},
            },
            cmap=centered_cmap(np.array([-np.max(occ1_df_for_pt['Agriculture\ngrowth path']), 
                                np.max(occ1_df_for_pt['Agriculture\ngrowth path'])]),
                               cmap=matplotlib.cm.RdBu_r, 
                               num_stds = 1,
#                                num_stds=np.max(occ1_df_for_pt['Agriculture\ngrowth path'])
                              ),
            group=group_dict['unemp'],
            formatter="{:.2f}",
        ),
        ColumnDefinition(
            name="Manufacturing\ngrowth path",
#             width=0.75,
            textprops={
                "ha": "center",
                "bbox": {"boxstyle": "round", "pad": 0.35, "alpha":alph},
                
            },
            cmap=centered_cmap(np.array([-np.max(occ1_df_for_pt['Agriculture\ngrowth path']), 
                                np.max(occ1_df_for_pt['Agriculture\ngrowth path'])]),
                               cmap=matplotlib.cm.RdBu_r, 
                               num_stds = 1,
#                                num_stds=np.max(occ1_df_for_pt['Agriculture\ngrowth path'])
                              ),
            group=group_dict['unemp'],
                        border="left",
            formatter="{:.2f}",
        ),
    ]
    + [
        ColumnDefinition(
            name="reg\nagr",
#             width=0.75,
            textprops={
                "ha": "center",
                "bbox": {"boxstyle": "round", "pad": 0.35, "alpha":alph},
            },
            cmap=centered_cmap(np.array([np.min(occ1_df_for_pt['reg\nagr']), 
                                -np.min(occ1_df_for_pt['reg\nagr'])]),
                               cmap=matplotlib.cm.PuOr, 
                               num_stds = 1,
#                                num_stds=-np.min(occ1_df_for_pt['reg\nagr'])
                              ),
            group=group_dict['vacs'],
            formatter="{:.2f}",
        ),
        ColumnDefinition(
            name="reg\nmanuf",
#             width=0.75,
            textprops={
                "ha": "center",
                "bbox": {"boxstyle": "round", "pad": 0.35, "alpha":alph},
            },
            cmap=centered_cmap(np.array([np.min(occ1_df_for_pt['reg\nagr']), 
                                -np.min(occ1_df_for_pt['reg\nagr'])]),
                               cmap=matplotlib.cm.PuOr, 
                               num_stds = 1,
#                                num_stds=-np.min(occ1_df_for_pt['reg\nagr'])
                              ),
            group=group_dict['vacs'],
                        border="left",

            formatter="{:.2f}",
        ),
    ]
)

In [ ]:
save_table = True #True #False

In [ ]:
fig, ax = plt.subplots(figsize=(9, 5)) ##18, 10

table = Table(
    occ1_df_for_pt.sort_values('Mean wage\n(BRL)', ascending=False),
    column_definitions=col_defs,
    row_dividers=True,
    footer_divider=True,
    ax=ax,
    textprops={"fontsize": 10}, ##14
    cell_kw = {"alpha": 0.5},
    row_divider_kw={"linewidth": 1, "linestyle": (0, (1, 5))},
    col_label_divider_kw={"linewidth": 1, "linestyle": "-"},
    column_border_kw={"linewidth": 1, "linestyle": "-"},
).autoset_fontcolors()#colnames=["OFF", "DEF"])

# plt.annotate('here', (0,0))
if save_table == True:
    fig.savefig("figures/occ1_table_results.pdf", facecolor=ax.get_facecolor(), dpi=200, bbox_inches='tight')
    fig.savefig("figures/occ1_table_results.png", facecolor=ax.get_facecolor(), dpi=200, bbox_inches='tight')

In [ ]:
plt.scatter(range(10), occ1_df_for_pt['Agriculture\ngrowth path'], c=occ1_df_for_pt['Agriculture\ngrowth path'],
           cmap='RdBu_r', vmin=-np.max(occ1_df_for_pt['Agriculture\ngrowth path']))

plt.colorbar(label='2018-2030 Average unemployment\nrate change from baseline (%-point)')
if save_table == True:
    plt.savefig('figures/unemp_colorbar_for_occ1table.pdf', bbox_inches='tight')
    plt.savefig('figures/unemp_colorbar_for_occ1table.png', bbox_inches='tight')
plt.show()

In [ ]:
plt.scatter(range(10), occ1_df_for_pt['reg\nagr'], c=occ1_df_for_pt['reg\nagr'],
           cmap='PuOr', vmax=-np.min(occ1_df_for_pt['reg\nagr']))
plt.colorbar(label='2018-2030 Average 6-month unfilled\nvacancy rate change from baseline (%-point)')
if save_table == True:
    plt.savefig('figures/vacs_colorbar_for_occ1table.pdf', bbox_inches='tight')
    plt.savefig('figures/vacs_colorbar_for_occ1table.png', bbox_inches='tight')
plt.show()

## Scenarios 

In [ ]:
save

In [ ]:
scn = scenarios[1:2][0]
jobs_created_0 = np.sum(np.array((scenario_dict_norm_to_bl[scn].loc[2030] - 
                           scenario_dict_norm_to_bl['Cen2BAseq'].loc[2030]).values)[np.where(x_data_0>0)[0]])
jobs_lost_0 = np.sum(np.array((scenario_dict_norm_to_bl[scn].loc[2030] - 
                           scenario_dict_norm_to_bl['Cen2BAseq'].loc[2030]).values)[np.where(x_data_0<=0)[0]])

scn = scenarios[2:3][0]
jobs_created = np.sum(np.array((scenario_dict_norm_to_bl[scn].loc[2030] - 
                           scenario_dict_norm_to_bl['Cen2BAseq'].loc[2030]).values)[np.where(x_data>0)[0]])
jobs_lost = np.sum(np.array((scenario_dict_norm_to_bl[scn].loc[2030] - 
                           scenario_dict_norm_to_bl['Cen2BAseq'].loc[2030]).values)[np.where(x_data<=0)[0]])


In [ ]:
print('jobs created agr', jobs_created_0)
print('jobs lost agr', jobs_lost_0)

print('jobs created manuf', jobs_created)
print('jobs lost manuf', jobs_lost)

In [ ]:
scn = scenarios[1:2][0]
jobs_created_0_large = np.sum(np.array((scenario_dict_norm_to_bl[scn].loc[2030] - 
                           scenario_dict_norm_to_bl['Cen2BAseq'].loc[2030]).iloc[large_occs].values)[
                                np.where(x_data_0.iloc[large_occs]>0)[0]])
jobs_lost_0_large = np.sum(np.array((scenario_dict_norm_to_bl[scn].loc[2030] - 
                           scenario_dict_norm_to_bl['Cen2BAseq'].loc[2030]).iloc[large_occs].values)[
                                np.where(x_data_0.iloc[large_occs]<=0)[0]])

scn = scenarios[2:3][0]
jobs_created_large = np.sum(np.array((scenario_dict_norm_to_bl[scn].loc[2030] - 
                           scenario_dict_norm_to_bl['Cen2BAseq'].loc[2030]).iloc[large_occs].values)[
                                np.where(x_data_0.iloc[large_occs]>0)[0]])

jobs_lost_large = np.sum(np.array((scenario_dict_norm_to_bl[scn].loc[2030] - 
                           scenario_dict_norm_to_bl['Cen2BAseq'].loc[2030]).iloc[large_occs].values)[
                                np.where(x_data_0.iloc[large_occs]<=0)[0]])


In [ ]:
print('jobs created agr', jobs_created_0_large)
print('jobs lost agr', jobs_lost_0_large)

print('jobs created manuf', jobs_created_large)
print('jobs lost manuf', jobs_lost_large)

In [ ]:
scn = scenarios[1:2][0]
## get entries where x_data_0 > 0 and per 1-digit ocupation 
## sum for each occupation
decrease_demand_0 = np.zeros(10)
increase_demand_0 = np.zeros(10)
for onedigit_occ in range(10):
#     print(onedigit_occ)
    increase_demand_0[onedigit_occ] = np.sum(np.array((scenario_dict_norm_to_bl[scn].loc[2030] - 
                           scenario_dict_norm_to_bl['Cen2BAseq'].loc[2030]).iloc[large_occs].values)[
            list(np.where((results_df_large_occs_only['occ_1']==onedigit_occ)&(
                results_df_large_occs_only['x_agr']>0))[0])])
    
    decrease_demand_0[onedigit_occ] = np.sum(np.array((scenario_dict_norm_to_bl[scn].loc[2030] - 
                           scenario_dict_norm_to_bl['Cen2BAseq'].loc[2030]).iloc[large_occs].values)[
            list(np.where((results_df_large_occs_only['occ_1']==onedigit_occ)&(
                results_df_large_occs_only['x_agr']<0))[0])])

In [ ]:
save = True

In [ ]:
f, ax = plt.subplots(figsize=(3,2.5))
# f.set_figwidth(2.5)

ax.barh(np.array(range(1,10)), increase_demand_0[row_idxs.iloc[::-1]][:9], color='tab:blue', zorder=2)
ax.barh(np.array(range(1,10)), decrease_demand_0[row_idxs.iloc[::-1]][:9], color='tab:red', zorder=2)

# ax.set_title('Agriculture growth path')
ax.set_xlabel('Total increase and decrease\nin jobs from 2018 to 2030\nper occupational group', fontsize=10)
ax.set_yticks(np.array(range(1,10)), '', fontsize=10)

# ax.set_xscale('symlog')
ylim = ax.get_ylim()
ax.plot([0,0],[0,10], linewidth=0.5, c='black')
ax.set_ylim(ylim)
ax.grid(which='both', zorder=0)


xticks_major = np.arange(-2e6, 3.1e6, 1e6)
plt.xticks(xticks_major)
ax.xaxis.set_major_locator(plt.MultipleLocator(1000000))
ax.xaxis.set_minor_locator(plt.MultipleLocator(500000))



# ax.yaxis.set_major_locator(plt.MultipleLocator(1))
t = ax.xaxis.get_offset_text()
t.set_x(1.1)



if save == True:
    plt.savefig('figures/labour_demand_agr.pdf', bbox_inches='tight')
    plt.savefig('figures/labour_demand_agr.png', bbox_inches='tight')

plt.show()

In [ ]:
scn = scenarios[2:3][0]
## get entries where x_data_0 > 0 and per 1-digit ocupation 
## sum for each occupation
decrease_demand = np.zeros(10)
increase_demand = np.zeros(10)
for onedigit_occ in range(0,10):
#     print(onedigit_occ)
    increase_demand[onedigit_occ] = np.sum(np.array((scenario_dict_norm_to_bl[scn].loc[2030] - 
                           scenario_dict_norm_to_bl['Cen2BAseq'].loc[2030]).iloc[large_occs].values)[
            list(np.where((results_df_large_occs_only['occ_1']==onedigit_occ)&(
                results_df_large_occs_only['x_manuf']>0))[0])])
    
    decrease_demand[onedigit_occ] = np.sum(np.array((scenario_dict_norm_to_bl[scn].loc[2030] - 
                           scenario_dict_norm_to_bl['Cen2BAseq'].loc[2030]).iloc[large_occs].values)[
            list(np.where((results_df_large_occs_only['occ_1']==onedigit_occ)&(
                results_df_large_occs_only['x_manuf']<0))[0])])

In [ ]:
cbo_1

In [ ]:
f, ax = plt.subplots(figsize=(3,2.5))


ax.barh(np.array(range(1,10)), increase_demand[row_idxs.iloc[::-1]][:9], color='tab:blue', zorder=2)
ax.barh(np.array(range(1,10)), decrease_demand[row_idxs.iloc[::-1]][:9], color='tab:red', zorder=2)

# ax.set_title('Manufacturing growth path')
ax.set_xlabel('Total increase and decrease\nin jobs from 2018 to 2030\nper occupational group', fontsize=10)
ax.set_yticks(np.array(range(1,10)), cbo_1['TITLE_short'][row_idxs.iloc[::-1]][:9])
# ax.set_xscale('symlog')
ylim = ax.get_ylim()
ax.plot([0,0],[0,10], linewidth=0.5, c='black')
ax.set_ylim(ylim)
ax.grid(which='both', zorder=0)

# ax.xaxis.set_major_locator(plt.MultipleLocator(200000))
# ax.yaxis.set_major_locator(plt.MultipleLocator(1))

# ax.xaxis.set_minor_locator(plt.MultipleLocator(100000))


xticks_major = np.arange(-500000, 500001, 500000)
plt.xticks(xticks_major)
# plt.ticklabel_format(axis='x', style='sci',scilimits=(6,8))


if save == True:
    plt.savefig('figures/labour_demand_manuf.pdf', bbox_inches='tight')
    plt.savefig('figures/labour_demand_manuf.png', bbox_inches='tight')

plt.show()

## Assortativity

In [ ]:
occs_570 = np.sort(results_df['occ_code'].unique())

In [ ]:
omng_occs = omng_df.copy()

In [ ]:
omng_occs.index = ([occ[:4] for occ in list(omng_df.index)])
omng_occs.columns = ([occ[:4] for occ in list(omng_df.index)])

In [ ]:
omng_occs

In [ ]:
omng_occs = omng_occs.groupby(omng_occs.index).sum()

In [ ]:
omng_occs = omng_occs.T.groupby(omng_occs.T.index).sum()

In [ ]:
omng_occs

In [ ]:
all(occs_570 == omng_occs.index)

In [ ]:
occs_570_1digit = pd.DataFrame({'occ_1': [str(occ[0]) for occ in occs_570]})


In [ ]:
# function that gives assortativity coefficient
# Joris Bücker, R Maria del Rio-Chanona, Anton Pichler, Matthew C Ives, and J Doyne Farmer. 
# Employment dynamics in a rapid decarbonization of the US power sector. 
# Joule, 2025

def delta_kron(i,j):
    if i == j:
        return 1
    else:
        return 0


def assortativity_weighted_directed_v2(A, x):
    """
    Computes assortativity coefficient for weighted and directed networks
    A(np array): adjacency matrix of network
    x(np array): nodes' attributes
    """
    n = A.shape[0]
    W = A.sum()
    W_div = 1/W
    # outstregth
    s_plus = A.sum(axis=1)
    # instrength
    s_minus = A.sum(axis=0)
    # average over in and out str
    mu_plus = (s_plus * x).sum() * W_div
    mu_minus = (s_minus * x).sum() * W_div
    # naming covariance and sf for + and -
    cov = 0
    sd_plus = 0
    sd_minus = 0
    for i in range(n):
        for j in range(n):
            cov += A[i,j] * (x[i] - mu_plus) * (x[j] - mu_minus)
            sd_plus += A[i,j] * (x[i] - mu_plus)** 2
            sd_minus += A[i,j] * (x[i] - mu_minus)** 2
    return cov / np.sqrt(sd_plus * sd_minus)



In [ ]:
print(assortativity_weighted_directed_v2(np.array(omng_occs), occs_570_1digit['occ_1'].astype(int)))

In [ ]:
print(assortativity_weighted_directed_v2(A_omn, results_df['occ_1'].astype(int)))

In [ ]:
print(assortativity_weighted_directed_v2(A_omn, results_df['loc_code'].astype(int)))

In [ ]:
## reg, occ, 1-digit network: 0.68, 0.50, 0.70